In [1]:
import os
import pickle
import string

SAVEDICTIONARYTOPICKLEFILE = 1
USEPICKLEFILE = 1

REBUILDDICTIONARYFROMPICKLEFILE = 0

path = 'F:\LaTeX\LatexWork\Bible'
os.chdir(path)
print (os.getcwd())

import plotly

F:\LaTeX\LatexWork\Bible


In [2]:
# %load BibleWordList2.py
'''
Update 21 Deceber 2018, 0004
'''


import os
import pickle
import collections
import operator
import re

DEBUG = 0

SAVEDICTIONARYTOPICKLEFILE = 1
USEPICKLEFILE = 1
REBUILDDICTIONARYFROMPICKLEFILE = 0

punctuations = '''!()_;:,<>?@#$^.'''

exceptions = [
    'A', 'Accept', 'After', 'Aha', 'All','Alas','Also','Arise',
    'Amen', 'An', 'And', 'As', 'At', 'Awake','According',
    'Be', 'Because', 'Before', 'Behold', 'Blessed','Build','Better',
    'Blessings', 'Blind', 'Both', 'Bring', 'But','Beautiful',
    'By', 'Command', 'Concerning', 'Consider', 'Cut','Come',"Children's",
    'Deal', 'Deceit', 'Deep', 'Deliver', 'Depart','Did',
    'Doubtless', 'Each', 'Either', 'Even', 'Every','Excellent','Fear',
    'For', 'From', 'Get', 'Give', 'Go', 'Grace',
    'Great', 'Hatred', 'Having', 'He', 'Hear',
    'Hearing', 'Heaviness', 'Help', 'His', 'Hold',
    'Horror', 'How', 'If', 'In', 'Incline', 'Judge',
    'Leave', 'Let', 'Lo', 'Look', 'Lying', 'Lodge','Make','Must','Mark',
    'Many', 'Mine', 'My', 'Neither', 'Nevertheless','Nay',
    'Not', 'Now', 'Number', 'O', 'Of', 'On', 'One','Out',
    'Only', 'Open', 'Or', 'Order', 'Our', 'Pardon',
    'Plead', 'Princes', 'Quicken', 'Red', 'Remember',
    'Remove', 'Riches', 'Righteous', 'Rise', 'Rivers','Return',
    'Sacrifice', 'Salvation', 'Say', 'Seven', 'Shall','Spake',
    'So', 'Speak', 'Sprinkle', 'Stablish', 'Stand','Strong',
    'Surely', 'Take', 'Teach', 'That', 'The', 'Their',
    'Then', 'There', 'Therefore', 'These', 'They',
    'This', 'Those', 'Thou', 'Though', 'Through',
    'Thus', 'Thy', 'To', 'Treasures', 'Trouble','Told',
    'Turn', 'Two', 'Unless', 'Uphold', 'We',
    'What', 'Whatsoever', 'When', 'Where', 'Wherefore',
    'Wherewithal', 'Which', 'Who', 'Whom', 'Whoso',
    'Whosoever', 'Why', 'Wise', 'With', 'Withhold','Walk','Wisdom',
    'Would', 'Ye', 'Yea', 'Yet', 'Your']

properNameList = [
    'Aaron', 'Abelmizraim', 'Abiasaph', 'Abib', 'Abihu', 'Abraham', 'Abram',
    'Aceldama', 'Achaia', 'Achaicus', 'Achor', 'Achzib', 'Adam', 'Adramyttium',
    'Adria', 'Adullam', 'Agrippa', 'Ahab', 'Ahaz', 'Ahikam', 'Ahisamach',
    'Aholiab', 'Alexandria', 'Alexandrians', 'Alphaeus', 'Amalek', 'Amittai',
    'Ammi ', 'Amminadab', 'Amon', 'Amorite', 'Amorites', 'Amram', 'Anathoth',
    'Andrew', 'Angel', 'Antioch', 'Aphrah', 'Apollos', 'Apphia', 'Aquila',
    'Arabia ', 'Arabian', 'Archippus', 'Ard', 'Areli', 'Aretas', 'Aristarchus',
    'Arnon', 'Arodi', 'Aroer ', 'Asenath', 'Asenath', 'Ashbel', 'Asher',
    'Ashkelon', 'Asia', 'Assir', 'Assyria ', 'Assyrian', 'Atad', 'Augustus',
    'Azariah', 'Azur', 'BRANCH', 'Baal ', 'Baali ', 'Baalim', 'Baalzephon',
    'Bablyon', 'Balaam', 'Balak', 'Barbarian', 'Barjona', 'Barnabas',
    'Barsabas', 'Bartholomew', 'Baruch', 'Becher', 'Beeri', 'Beerlahairoi',
    'Beersheba', 'Belah', 'Belial', 'Belteshazzar', 'Benjamin', 'Beor',
    'Bered', 'Beriah', 'Bernice', 'Bethaven', 'Bethdiblathaim ', 'Bethel',
    'Bethezel', 'Bethgamul', 'Bethlehem', 'Bethmeon', 'Bethshemesh',
    'Bezaleel', 'Bilhah', 'Bozrah', 'Caesar ', 'Caesarea', 'Cainan', 'Cana',
    'Canaan', 'Canaanite', 'Canaanites', 'Canaanitish', 'Capernaum', 'Caphtor',
    'Carchemish', 'Carmi', 'Castor', 'Cephas', 'Chaldeans', 'Chaldeans',
    'Chemosh', 'Chittim', 'Chloe', 'Christ', 'Cilicia', 'Clauda', 'Cnidus',
    'Crete', 'Cretians', 'Crispus', 'Cyprus', 'Cyrenians', 'Cyrus',
    'Damascenes', 'Damascus', 'Dan', 'Daniel', 'Darius', 'David', 'Demas',
    'Diblaim', 'Dibon', 'Didymus', 'Dinah', 'Edom', 'Egypt', 'Egyptian',
    'Egyptians', 'Ehi', 'Elasah', 'Eleazar', 'Elias', 'Eliezer', 'Elim',
    'Elisheba', 'Elkanah', 'Elon', 'Elzaphan', 'Enoch', 'Enos', 'Epaphras',
    'Ephesus', 'Ephraim', 'Ephrath', 'Ephron', 'Er', 'Eri', 'Esaias', 'Esau',
    'Etham', 'Euphrates', 'Euroclydon', 'Ezbon', 'Father', 'Felix', 'Festus',
    'Fortunatus', 'Gad', 'Gadarenes', 'Gaius', 'Galatia', 'Galilee', 'Gath',
    'Gaza', 'Gedaliah', 'Gemariah', 'Gentiles', 'Gera', 'Gershom', 'Gershon',
    'Gibeah', 'Gibeon', 'Gilead', 'Gilgal', 'God', 'Gomer', 'Goshen',
    'Grecians', 'Greek', 'Guni', 'Hagar', 'Haggi', 'Ham', 'Hananiah', 'Hanoch',
    'Heber', 'Hebrew', 'Hebrews', "Hebrews'", 'Hebron', 'Heldai', 'Helem',
    'Hen', 'Heshbon', 'Hezekiah', 'Hezron', 'High', 'Hilkiah', 'Hinnom',
    'Hittite', 'Hittites', 'Hivite', 'Hivites', 'Horeb', 'Horonaim', 'Hosea',
    'Hoshaiah', 'Huppim', 'Hur', 'Hushim', 'I', 'Isaac', 'Ishi', 'Ishmael',
    'Ishmeelites', 'Ishuah', 'Israel', 'Issachar', 'Issachar', 'Isui', 'Italy',
    'Ithamar', 'Ithiel', 'Izhar', 'JEHOVAH', 'Jachin', 'Jacob', 'Jahleel',
    'Jahzeel', 'Jairus', 'Jakeh', 'James', 'Jamin', 'Japheth', 'Jareb:',
    'Jared', 'Jebusite', 'Jebusites', 'Jeconiah', 'Jedaiah', 'Jehoiada',
    'Jehoiakim', 'Jehoshapha', 'Jehovahnissi', 'Jehu', 'Jemuel', 'Jeremiah',
    'Jerusalem', 'Jesus', 'Jethro', 'Jews', 'Jezer', 'Jezreel', 'Jimnah',
    'Joash', 'Job', 'Jochebed', 'Joel', 'Johanan', 'John', 'Jonah', 'Jonas',
    'Joppa', 'Jordan', 'Josedech', 'Joseph', 'Josiah', 'Jotham', 'Judaea',
    'Judah', 'Judas', 'Julius', 'Just ', 'Justus ', 'Kadesh', 'Kareah',
    'Kedar', 'Kerioth', 'King', 'Kiriathaim ', 'Kohath', 'Korah', 'Korhites',
    'LORD', 'Laban', 'Laban', 'Lachis', 'Lamech', 'Laodicea', 'Lasea', 'Leah',
    'Lebanon', 'Legion', 'Levi', 'Libertines', 'Libni', 'Libyans', 'Loammi',
    'Lord', 'Lord Jesus Christ', 'Loruhamah', 'Lucas', 'Luhith', 'Luz', 'Lycia',
    'Lydians', 'Maaseiah', 'Macedonia', 'Macedonian', 'Machir', 'Machpelah',
    'Mahalaleel', 'Mahali', 'Malachi', 'Malchiel', 'Mamre', 'Manasseh',
    'Marcus', 'Mareshah', 'Maroth', 'Massah', 'Matthew', 'Matthias',
    'Melchizedek', 'Melita', 'Melzar', 'Merari', 'Meribah', 'Messias',
    'Methuselah', 'Micah', 'Midian', 'Migdol', 'Miriam', 'Misgab', 'Mishael',
    'Mizpah', 'Moab', 'Morasthite', 'Moreshethgath', 'Moses', 'Muppim', 'Mushi',
    'Myra', 'Naaman', 'Naashon', 'Nadab', 'Naphtali', 'Nathanael', 'Nazareth',
    'Nebo', 'Nebuchadnezzar', 'Nebuchadrezzar', 'Nebuzaradan', 'Nehelamite',
    'Nepheg', 'Neriah', 'Nicanor', 'Nicolas', 'Nicopolis', 'Nineveh', 'No ',
    'Noah', 'Noph', 'Ohad', 'Olivet', 'Omri', 'Onan', 'Onesimus', 'Padan',
    'Padanaram', 'Palestine', 'Pallu', 'Pamphylia', 'Parmenas', 'Pashur',
    'Pathros', 'Paul', 'Pentecost', 'Pergamos', 'Perizzite', 'Perizzites',
    'Peter', 'Pethuel', 'Phallu', 'Pharaoh', "Pharaoh's", 'Pharaohhophra ',
    'Pharaohnecho', 'Pharez', 'Pharisee ', 'Pharisees', 'Phenice',
    'Philadelphia', 'Philemon', 'Philip', 'Philippi', 'Philistines',
    'Phinehas', 'Phuvah', 'Pihahiroth', 'Pilate', 'Pithom', 'Pollux',
    'Potiphar', 'Potipherah', 'Prince', 'Priscilla', 'Prochorus', 'Puah',
    'Publius', 'Puteoli', 'Putiel', 'Raamses', 'Rachel', 'Ramah', 'Rameses',
    'Rephidim', 'Reuben', 'Reuel', 'Rhegium', 'Rock', 'Rome', 'Rosh', 'Ruhamah',
    'Sabeans', 'Sadducees', 'Salem', 'Salmone', 'Samaria', 'Samaritans',
    'Samuel', 'Saphir', 'Sara', 'Sarah', 'Sarai', 'Sardis', 'Satan', 'Saul',
    'Saviour', 'Scythian', 'Selah', 'Sepharad', 'Serah', 'Sered', 'Seth',
    'Shaphan', 'Shaul', 'Shelah', 'Shem', 'Shemaiah', 'Shillem', 'Shiloh',
    'Shimi', 'Shimron', 'Shiphrah', 'Shittim', 'Shuni', 'Sidon', 'Sihon',
    'Sihor', 'Simeon', 'Simon ', 'Simon Zelotes', 'Sinai', 'Sirion', 'Smyrna',
    'Solomon', 'Sosthenes', 'Stephanas', 'Stephen', 'Succoth', 'Sychar',
    'Syracuse', 'Tabor', 'Tahpanhes', 'Tarshish', 'Teman', 'Theophilus',
    'Thessalonica', 'Thomas', 'Thummim', 'Thyatira', 'Tiberias', 'Timon',
    'Timotheus', 'Timothy', 'Titus', 'Tobijah', 'Tola', 'Tophet', 'Tychicus',
    'Tyre', 'Tyrus', 'Ucal', 'Uri', 'Urim', 'Uzziah', 'Uzziel', 'Zaanan',
    'Zaphnathpaaneah', 'Zarephath', 'Zebedee', 'Zebulun', 'Zedekiah',
    'Zephaniah', 'Zerah', 'Zichri', 'Zidon', 'Zilpah', 'Zion', 'Ziphion',
    'Zipporah', 'Zithri', 'Zohar']

##########
def removePunc(input):
    no_punc = ''
    for char in input:
        if char not in punctuations:
            no_punc = no_punc + char
    return no_punc


##########
def generatedLatexFormattedText(dict):
    BUILDAWIP = 1
    BUILDNWIV = 1
    BUILDPNIP = 1
    BUILDNWIC = 1
    BUILDNLIW = 1
    BUILDBPIP = 1
    MINISTRY = 1

    topString = "\\footnote{\\textcolor[cmyk]{0.99998,1,0,0}{\\hyperlink{TOC}{Return to end " \
                "of Table of Contents.}}}"
    textColorString = "\\textcolor[cmyk]{0.99998,1,0,0}{"
    endString = "}"

    '''
    Print plain text with Latex color header:
        \textcolor[rgb]{0.00,0.00,1.00} == BLUE
        \textcolor[cmyk]{0.99998,1,0,0} == A DEEPER BLUE
    '''
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('Chapter with Latex format\n%%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')

    linesWith13Words = []
    linesWith18Words = []
    for line in dict:
        booknm = line[0]
        chapnm = line[1]
        chapno = line[2]
        versno = line[3]
        versn2 = line[4]
        newline = line[5]
        if versn2 == '1':
            print(topString + textColorString + newline + endString)
        else:
            print('[' + versn2 + '] ' + textColorString + newline + endString)
        words = newline.split()
        wordLength = len(words)
        if wordLength == 13:
            linesWith13Words.append(line)
        if wordLength == 18:
            linesWith18Words.append(line)
            
        
    '''
    NOW print the verses with LaTeX index items:
        AWIP - All Words in Passage
        NWIV - NUmber of Words in Verse
        PNIP - Proper noun in passage
        ... more later
    '''
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('Chapter with indexed items\n%%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')
    for line in dict:
        booknm = line[0]
        chapnm = line[1]
        chapno = line[2]
        versno = line[3]
        versn2 = line[4]
        verseRef = "!" + booknm + "!" + chapnm + " " + chapno + ":" + versno 
        line = line[5]
        if versn2 == '1':
            verseString = topString + textColorString + line + endString
        else:
            verseString = '[' + versn2 + '] ' + textColorString + line + endString
        #print(booknm,chapnm,chapno,versno,versn2) 
        '''
        NWIV
        '''
        words = line.split()
        numWords = len(words)
        if DEBUG == 1:
            print(numWords,'Words in verse',versn2)
        extraString="\\index[NWIV]{" + str(numWords) + verseRef + "}"
        '''
        AWIP - here, Passage = verse
        '''
        verseDict = {}
        wordString = ''
        for word in words:
            word = removePunc(word)
            if word not in verseDict:
                verseDict[word] = 1
                wordString = wordString + "\\index[AWIP]{" + word + verseRef + "}"
            else:
                verseDict[word] = verseDict[word] + 1
                wordString = wordString + "\\index[AWIP]{" + word + verseRef 
                wordString = wordString + " (" + str(verseDict[word]) + ")}"
        '''
        PNIP - Proper names in passage
        '''
        propernameStrng = ''
        for item in verseDict:
            if item in properNameList:
                if item not in exceptions:
                    propernameStrng = propernameStrng + "\\index[PNIP]{" + item + verseRef + "}"

        builtVerseString = verseString + extraString + wordString  + propernameStrng
        print(builtVerseString)
    return linesWith18Words, linesWith13Words

##########
def showWordsByFrequency(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    print('\\subsection{Words by Frequency}')
    print ("\\scriptsize")
    print ("\\begin{center}")
    print ("\\begin{longtable}{l|r}")
    print ("\caption[Word Frequencies in "+m.group(1)+" "+str(int(m.group(2)))+"]{Word Frequencies in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    sortedDictionary = sorted(dictionary.items(), key=lambda x: x[1], reverse=True)
    for item in sortedDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
    print ("\\end{center}")
    print ("\n\n")
    print ("\\normalsize")
    print ("\n\n")    

##########
def showWordsAlphabetically(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    print('\\subsection{Words Alphabetically}')
    print ("\\scriptsize")
    print ("\\begin{center}")
    print ("\\begin{longtable}{l|r}")
    print ("\caption[Word Alphabetically in "+m.group(1)+" "+str(int(m.group(2)))+"]{Word Alphabetically in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    sortedDictionary = sorted(dictionary.items(), key=lambda x: x[0], reverse=False)
    for item in sortedDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
    print ("\\end{center}")
    print ("\n\n")
    print ("\\normalsize")
    print ("\n\n")    

##########
def generateWordStatisticsForChapter(label,verses):
    '''
    Need to simplify this code! ONLY count number of words, number of unique 
    words, number of itallic words, number of unique itallic words
    
    verseDictionary, rest for each verse
    chapterDictionary
    '''
    global properNameList
    wordCount = 0
    emphasizedwordCount = 0
    wordDict = {}
    emphasizedwordDict = {}
    verseString = ''
    A = re.findall(r'\d+', label)
    A = " ".join(map(str, A))
    digit = int(A)

    word1 = " ".join(re.findall("[a-zA-Z]+", label))
    
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('%%%%% Word Statistics\n%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')
    
    print('\\normalsize')

    verseStats = ["\\subsection{Chapter Word Statistics}",
                  "\n",
                  "%%%%%%%%%%",
                  "%%%%%%%%%%",
                  " ",
                  "\\begin{center}",
                  "\\begin{longtable}{l|c|c|c|c}",
                  "\\caption[Stats for "+word1+" "+str(digit) +"]{Stats for "+word1+" "+str(digit) +"} \\label{table:Stats for "+word1+" "+str(digit) +"} \\\ ",
                  "\\hline \multicolumn{1}{|c|}{\\textbf{Verse(s)}} & \multicolumn{1}{|c|}{\\textbf{Count}} & \multicolumn{1}{|c|}{\\textbf{Unique}} & \multicolumn{1}{|c|}{\\textbf{Italics}} & \multicolumn{1}{|c|}{\\textbf{Uniq Italic}}  \\\ \\hline ",
                  "\\endfirsthead",
                  " ",
                  "\multicolumn{5}{c}",
                  "{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\  ",
                  "\\hline \multicolumn{1}{|c|}{\\textbf{Verse(s)}} & \multicolumn{1}{|c|}{\\textbf{Count}} & \multicolumn{1}{|c|}{\\textbf{Unique}} & \multicolumn{1}{|c|}{\\textbf{Italics}} & \multicolumn{1}{|c|}{\\textbf{Uniq Italic}}  \\\ \\hline ",
                  "\\endhead",
                  " ",
                  "\\hline \\multicolumn{5}{|r|}{{Continued if needed}} \\\ \\hline",
                  "\\endfoot ",]

    '''
    This section prints JUST the Bible text in a LaTeX format with a textcolor encoding added
    Original Code
    '''
    chapterDict = {}
    chapwordWords = 0
    chapterUniqueWords = 0
    numberChapterUniqueItallics = 0
    numberChapterItallics = 0
    verseNo = 0
    for line in verses:
        verseNo += 1
        verseDict = {}
#        print(line)
        words = removePunc(line[5])
        wordcount = 0
        uniqueWords = 0        
        '''
        Process for verse
        '''
        for word in words.split():
            wordcount += 1
            if word not in verseDict:
                verseDict[word] = 1
                uniqueWords += 1
            else:
                verseDict[word] = verseDict[word] + 1
        numberUniqueItallics = 0
        numberItallics = 0
        for item in verseDict:
            if "\emph{" in item:
                numberUniqueItallics += 1
                numberItallics = numberItallics + verseDict[item]                
        '''
        Process for chapter
        '''                
        for word in words.split():
            chapwordWords += 1
            if word not in chapterDict:
                chapterDict[word] = 1
                chapterUniqueWords += 1
            else:
                chapterDict[word] = chapterDict[word] + 1
        sequence = str(verseNo),str(wordcount),str(uniqueWords),str(numberItallics),str(numberUniqueItallics)+'\\\ \hline'
        S = ' & '.join(sequence)
        verseStats.append(S)
    for item in chapterDict:
        if "\emph{" in item:
            numberChapterUniqueItallics +=1
            numberChapterItallics = numberChapterItallics + chapterDict[item]

    '''
    Finish Up!
    '''
#    print ("\n",wordCount,"words in passage, of which",len(wordDict),"are unique")
#    print (emphasizedwordCount,'words in ital, of which',len(emphasizedwordDict),'are unique')
    print ("\n\n")
    verseStats.append("\hline \hline")
    sequence = 'Total',str(chapwordWords),str(chapterUniqueWords),str(numberChapterItallics),str(numberChapterUniqueItallics)
    S = ' & '.join(sequence)
    verseStats.append(S)
    for item in verseStats:
        print (item)
    print ("\n\n")
    print ("\\end{longtable}")
    print ("\\end{center}\n")
#    print (chapwordWords,"words in passage, of which",chapterUniqueWords,"are unique")
#    print (numberChapterItallics,'words in ital, of which',numberChapterUniqueItallics,'are unique')
#    print ("\n\n")
#
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
    
    return chapterDict

##########
def getWordLengthsInChapter(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    wordLengthDictionary = {}
    for item in dictionary:
        if "\emph{" in item:
            tempitem = item.replace("\emph{",'')
            tempitem = tempitem.replace('}','')
            if len(tempitem) in wordLengthDictionary:
                wordLengthDictionary[len(tempitem)] = wordLengthDictionary[len(tempitem)] + ', ' + item
            else:
                    wordLengthDictionary[len(tempitem)] = item
        else:
            if len(item) in wordLengthDictionary:
                wordLengthDictionary[len(item)] = wordLengthDictionary[len(item)] + ', ' + item
            else:
                wordLengthDictionary[len(item)] = item
    wordLengthDictionary = sorted(wordLengthDictionary.items(), key=lambda x:x[0], reverse=False)

    print('\\subsection{Word Lengths in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Words by Length in "+m.group(1)+" "+str(int(m.group(2)))+"]{Words by Length in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Length}} & \multicolumn{1}{c|}{\\textbf{Words}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Length}} & \multicolumn{1}{c|}{\\textbf{Words}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in wordLengthDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")

##########
def makeVersesWith13WordsTable(list,arg):
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")

    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)


    print('\\subsection{Verses with 13 Words in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Verses with 13 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"]{Verses with 13 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:Verses with 13 Words in-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in list:
        print (item[0]+' '+item[2]+":"+item[4], "&", item[5], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
        
##########
          
##########
def makeVersesWith18WordsTable(list,arg):
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")

    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)


    print('\\subsection{Verses with 18 Words in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Verses with 18 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"]{Verses with 18 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:Verses with 18 Words in-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in list:
        print (item[0]+' '+item[2]+":"+item[4], "&", item[5], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
        
##########

def countLettersInVerses(arg1,arg2):
    for item in arg2:
        letterDict = {}
        for item2 in item[5]:
            if "\emph" in item2:
                item2 = item2.replace('\emph','')
            if item2 != ' ':
                if item2 in letterDict:
                    letterDict[item2] = letterDict[item2] + 1
                else:
                    letterDict[item2] =  1
            print ("for verse",item[2],":",letterDict)

def ProcessVerses(arg1,arg2):
    ''' 
    arg1 is dictionary name and arg2 is the dictionary
    '''
#    saveToPickleFile(arg1,arg2)  
    linesWith18words, linesWith13words = generatedLatexFormattedText(arg2)         # format chapter in Latex
    #linesWith13words is for use later ... 
    chapterDict = generateWordStatisticsForChapter(arg1,arg2)   # do chapter word statistics
    showWordsByFrequency(chapterDict,arg1)                      # show words by frequency
    showWordsAlphabetically(chapterDict,arg1)                   # show words alphabetically
    getWordLengthsInChapter(chapterDict,arg1)                   # show word lengths in chapter
    
    if len(linesWith13words) > 0:
        #print(linesWith13words)
        makeVersesWith13WordsTable(linesWith13words,arg1)
        
    if len(linesWith18words) > 0:
        #print(linesWith13words)
        makeVersesWith18WordsTable(linesWith18words,arg1)
        
    ''' In Development
    countLettersInVerses(arg1,arg2)
    '''



In [3]:
path = 'F:\python'
os.chdir(path)
print (os.getcwd())

F:\python


In [4]:
# %load NEW12OT-MakeConcordance-2Kings.py
"""
Created on Sun Jan 01 07:52:18 2017

@author: keith
"""


import os
import pickle
import string

from BibleWordList2 import *

BUILDAWIP = 1
BUILDNWIV = 1
BUILDPNIP = 1
BUILDNWIC = 1
BUILDNLIW = 1
BUILDBPIP = 1

SAVEDICTIONARYTOPICKLEFILE = 1
USEPICKLEFILE = 1
REBUILDDICTIONARYFROMPICKLEFILE = 0

SecondKings01 = [
    ['2Kings','2Ki','01','001','1',"Then Moab rebelled against Israel after the death of Ahab."],
['2Kings','2Ki','01','002','2',"And Ahaziah fell down through a lattice in his upper chamber that \emph{was} in Samaria, and was sick: and he sent messengers, and said unto them, Go, enquire of Baal-zebub the god of Ekron whether I shall recover of this disease."],
['2Kings','2Ki','01','003','3',"But the angel of the LORD said to Elijah the Tishbite, Arise, go up to meet the messengers of the king of Samaria, and say unto them, \emph{Is} \emph{it} not because \emph{there} \emph{is} not a God in Israel, \emph{that} ye go to enquire of Baal-zebub the god of Ekron?"],
['2Kings','2Ki','01','004','4',"Now therefore thus saith the LORD, Thou shalt not come down from that bed on which thou art gone up, but shalt surely die. And Elijah departed."],
['2Kings','2Ki','01','005','5',"And when the messengers turned back unto him, he said unto them, Why are ye now turned back?"],
['2Kings','2Ki','01','006','6',"And they said unto him, There came a man up to meet us, and said unto us, Go, turn again unto the king that sent you, and say unto him, Thus saith the LORD, \emph{Is} \emph{it} not because \emph{there} \emph{is} not a God in Israel, \emph{that} thou sendest to enquire of Baal-zebub the god of Ekron? therefore thou shalt not come down from that bed on which thou art gone up, but shalt surely die."],
['2Kings','2Ki','01','007','7',"And he said unto them, What manner of man \emph{was} \emph{he} which came up to meet you, and told you these words?"],
['2Kings','2Ki','01','008','8',"And they answered him, \emph{He} \emph{was} an hairy man, and girt with a girdle of leather about his loins. And he said, It \emph{is} Elijah the Tishbite."],
['2Kings','2Ki','01','009','9',"Then the king sent unto him a captain of fifty with his fifty. And he went up to him: and, behold, he sat on the top of an hill. And he spake unto him, Thou man of God, the king hath said, Come down."],
['2Kings','2Ki','01','010','10',"And Elijah answered and said to the captain of fifty, If I \emph{be} a man of God, then let fire come down from heaven, and consume thee and thy fifty. And there came down fire from heaven, and consumed him and his fifty."],
['2Kings','2Ki','01','011','11',"Again also he sent unto him another captain of fifty with his fifty. And he answered and said unto him, O man of God, thus hath the king said, Come down quickly."],
['2Kings','2Ki','01','012','12',"And Elijah answered and said unto them, If I \emph{be} a man of God, let fire come down from heaven, and consume thee and thy fifty. And the fire of God came down from heaven, and consumed him and his fifty."],
['2Kings','2Ki','01','013','13',"And he sent again a captain of the third fifty with his fifty. And the third captain of fifty went up, and came and fell on his knees before Elijah, and besought him, and said unto him, O man of God, I pray thee, let my life, and the life of these fifty thy servants, be precious in thy sight."],
['2Kings','2Ki','01','014','14',"Behold, there came fire down from heaven, and burnt up the two captains of the former fifties with their fifties: therefore let my life now be precious in thy sight."],
['2Kings','2Ki','01','015','15',"And the angel of the LORD said unto Elijah, Go down with him: be not afraid of him. And he arose, and went down with him unto the king."],
['2Kings','2Ki','01','016','16',"And he said unto him, Thus saith the LORD, Forasmuch as thou hast sent messengers to enquire of Baal-zebub the god of Ekron, \emph{is} \emph{it} not because \emph{there} \emph{is} no God in Israel to enquire of his word? therefore thou shalt not come down off that bed on which thou art gone up, but shalt surely die."],
['2Kings','2Ki','01','017','17',"So he died according to the word of the LORD which Elijah had spoken. And Jehoram reigned in his stead in the second year of Jehoram the son of Jehoshaphat king of Judah; because he had no son."],
['2Kings','2Ki','01','018','18',"Now the rest of the acts of Ahaziah which he did, \emph{are} they not written in the book of the chronicles of the kings of Israel?"]
]

SecondKings02 = [
    ['2Kings','2Ki','02','001','1',"And it came to pass, when the LORD would take up Elijah into heaven by a whirlwind, that Elijah went with Elisha from Gilgal."],
['2Kings','2Ki','02','002','2',"And Elijah said unto Elisha, Tarry here, I pray thee; for the LORD hath sent me to Beth-el. And Elisha said \emph{unto} \emph{him,} \emph{As} the LORD liveth, and \emph{as} thy soul liveth, I will not leave thee. So they went down to Beth-el."],
['2Kings','2Ki','02','003','3',"And the sons of the prophets that \emph{were} at Beth-el came forth to Elisha, and said unto him, Knowest thou that the LORD will take away thy master from thy head to day? And he said, Yea, I know \emph{it}; hold ye your peace."],
['2Kings','2Ki','02','004','4',"And Elijah said unto him, Elisha, tarry here, I pray thee; for the LORD hath sent me to Jericho. And he said, \emph{As} the LORD liveth, and \emph{as} thy soul liveth, I will not leave thee. So they came to Jericho."],
['2Kings','2Ki','02','005','5',"And the sons of the prophets that \emph{were} at Jericho came to Elisha, and said unto him, Knowest thou that the LORD will take away thy master from thy head to day? And he answered, Yea, I know \emph{it}; hold ye your peace."],
['2Kings','2Ki','02','006','6',"And Elijah said unto him, Tarry, I pray thee, here; for the LORD hath sent me to Jordan. And he said, \emph{As} the LORD liveth, and \emph{as} thy soul liveth, I will not leave thee. And they two went on."],
['2Kings','2Ki','02','007','7',"And fifty men of the sons of the prophets went, and stood to view afar off: and they two stood by Jordan."],
['2Kings','2Ki','02','008','8',"And Elijah took his mantle, and wrapped \emph{it} together, and smote the waters, and they were divided hither and thither, so that they two went over on dry ground."],
['2Kings','2Ki','02','009','9',"And it came to pass, when they were gone over, that Elijah said unto Elisha, Ask what I shall do for thee, before I be taken away from thee. And Elisha said, I pray thee, let a double portion of thy spirit be upon me."],
['2Kings','2Ki','02','010','10',"And he said, Thou hast asked a hard thing: \emph{nevertheless}, if thou see me \emph{when} \emph{I} \emph{am} taken from thee, it shall be so unto thee; but if not, it shall not be \emph{so}."],
['2Kings','2Ki','02','011','11',"And it came to pass, as they still went on, and talked, that, behold, \emph{there} \emph{appeared} a chariot of fire, and horses of fire, and parted them both asunder; and Elijah went up by a whirlwind into heaven."],
['2Kings','2Ki','02','012','12',"And Elisha saw \emph{it}, and he cried, My father, my father, the chariot of Israel, and the horsemen thereof. And he saw him no more: and he took hold of his own clothes, and rent them in two pieces."],
['2Kings','2Ki','02','013','13',"He took up also the mantle of Elijah that fell from him, and went back, and stood by the bank of Jordan;"],
['2Kings','2Ki','02','014','14',"And he took the mantle of Elijah that fell from him, and smote the waters, and said, Where \emph{is} the LORD God of Elijah? and when he also had smitten the waters, they parted hither and thither: and Elisha went over."],
['2Kings','2Ki','02','015','15',"And when the sons of the prophets which \emph{were} to view at Jericho saw him, they said, The spirit of Elijah doth rest on Elisha. And they came to meet him, and bowed themselves to the ground before him."],
['2Kings','2Ki','02','016','16',"And they said unto him, Behold now, there be with thy servants fifty strong men; let them go, we pray thee, and seek thy master: lest peradventure the Spirit of the LORD hath taken him up, and cast him upon some mountain, or into some valley. And he said, Ye shall not send."],
['2Kings','2Ki','02','017','17',"And when they urged him till he was ashamed, he said, Send. They sent therefore fifty men; and they sought three days, but found him not."],
['2Kings','2Ki','02','018','18',"And when they came again to him, (for he tarried at Jericho,) he said unto them, Did I not say unto you, Go not?"],
['2Kings','2Ki','02','019','19',"And the men of the city said unto Elisha, Behold, I pray thee, the situation of this city \emph{is} pleasant, as my lord seeth: but the water \emph{is} naught, and the ground barren."],
['2Kings','2Ki','02','020','20',"And he said, Bring me a new cruse, and put salt therein. And they brought \emph{it} to him."],
['2Kings','2Ki','02','021','21',"And he went forth unto the spring of the waters, and cast the salt in there, and said, Thus saith the LORD, I have healed these waters; there shall not be from thence any more death or barren \emph{land}."],
['2Kings','2Ki','02','022','22',"So the waters were healed unto this day, according to the saying of Elisha which he spake."],
['2Kings','2Ki','02','023','23',"And he went up from thence unto Beth-el: and as he was going up by the way, there came forth little children out of the city, and mocked him, and said unto him, Go up, thou bald head; go up, thou bald head."],
['2Kings','2Ki','02','024','24',"And he turned back, and looked on them, and cursed them in the name of the LORD. And there came forth two she bears out of the wood, and tare forty and two children of them."],
['2Kings','2Ki','02','025','25',"And he went from thence to mount Carmel, and from thence he returned to Samaria."]
]

SecondKings03 = [
    ['2Kings','2Ki','03','001','1',"Now Jehoram the son of Ahab began to reign over Israel in Samaria the eighteenth year of Jehoshaphat king of Judah, and reigned twelve years."],
['2Kings','2Ki','03','002','2',"And he wrought evil in the sight of the LORD; but not like his father, and like his mother: for he put away the image of Baal that his father had made."],
['2Kings','2Ki','03','003','3',"Nevertheless he cleaved unto the sins of Jeroboam the son of Nebat, which made Israel to sin; he departed not therefrom."],
['2Kings','2Ki','03','004','4',"And Mesha king of Moab was a sheepmaster, and rendered unto the king of Israel an hundred thousand lambs, and an hundred thousand rams, with the wool."],
['2Kings','2Ki','03','005','5',"But it came to pass, when Ahab was dead, that the king of Moab rebelled against the king of Israel."],
['2Kings','2Ki','03','006','6',"And king Jehoram went out of Samaria the same time, and numbered all Israel."],
['2Kings','2Ki','03','007','7',"And he went and sent to Jehoshaphat the king of Judah, saying, The king of Moab hath rebelled against me: wilt thou go with me against Moab to battle? And he said, I will go up: I \emph{am} as thou \emph{art}, my people as thy people, \emph{and} my horses as thy horses."],
['2Kings','2Ki','03','008','8',"And he said, Which way shall we go up? And he answered, The way through the wilderness of Edom."],
['2Kings','2Ki','03','009','9',"So the king of Israel went, and the king of Judah, and the king of Edom: and they fetched a compass of seven days' journey: and there was no water for the host, and for the cattle that followed them."],
['2Kings','2Ki','03','010','10',"And the king of Israel said, Alas! that the LORD hath called these three kings together, to deliver them into the hand of Moab!"],
['2Kings','2Ki','03','011','11',"But Jehoshaphat said, \emph{Is} \emph{there} not here a prophet of the LORD, that we may enquire of the LORD by him? And one of the king of Israel's servants answered and said, Here \emph{is} Elisha the son of Shaphat, which poured water on the hands of Elijah."],
['2Kings','2Ki','03','012','12',"And Jehoshaphat said, The word of the LORD is with him. So the king of Israel and Jehoshaphat and the king of Edom went down to him."],
['2Kings','2Ki','03','013','13',"And Elisha said unto the king of Israel, What have I to do with thee? get thee to the prophets of thy father, and to the prophets of thy mother. And the king of Israel said unto him, Nay: for the LORD hath called these three kings together, to deliver them into the hand of Moab."],
['2Kings','2Ki','03','014','14',"And Elisha said, \emph{As} the LORD of hosts liveth, before whom I stand, surely, were it not that I regard the presence of Jehoshaphat the king of Judah, I would not look toward thee, nor see thee."],
['2Kings','2Ki','03','015','15',"But now bring me a minstrel. And it came to pass, when the minstrel played, that the hand of the LORD came upon him."],
['2Kings','2Ki','03','016','16',"And he said, Thus saith the LORD, Make this valley full of ditches."],
['2Kings','2Ki','03','017','17',"For thus saith the LORD, Ye shall not see wind, neither shall ye see rain; yet that valley shall be filled with water, that ye may drink, both ye, and your cattle, and your beasts."],
['2Kings','2Ki','03','018','18',"And this is \emph{but} a light thing in the sight of the LORD: he will deliver the Moabites also into your hand."],
['2Kings','2Ki','03','019','19',"And ye shall smite every fenced city, and every choice city, and shall fell every good tree, and stop all wells of water, and mar every good piece of land with stones."],
['2Kings','2Ki','03','020','20',"And it came to pass in the morning, when the meat offering was offered, that, behold, there came water by the way of Edom, and the country was filled with water."],
['2Kings','2Ki','03','021','21',"And when all the Moabites heard that the kings were come up to fight against them, they gathered all that were able to put on armour, and upward, and stood in the border."],
['2Kings','2Ki','03','022','22',"And they rose up early in the morning, and the sun shone upon the water, and the Moabites saw the water on the other side \emph{as} red as blood:"],
['2Kings','2Ki','03','023','23',"And they said, This \emph{is} blood: the kings are surely slain, and they have smitten one another: now therefore, Moab, to the spoil."],
['2Kings','2Ki','03','024','24',"And when they came to the camp of Israel, the Israelites rose up and smote the Moabites, so that they fled before them: but they went forward smiting the Moabites, even in \emph{their} country."],
['2Kings','2Ki','03','025','25',"And they beat down the cities, and on every good piece of land cast every man his stone, and filled it; and they stopped all the wells of water, and felled all the good trees: only in Kir-haraseth left they the stones thereof; howbeit the slingers went about \emph{it}, and smote it."],
['2Kings','2Ki','03','026','26',"And when the king of Moab saw that the battle was too sore for him, he took with him seven hundred men that drew swords, to break through \emph{even} unto the king of Edom: but they could not."],
['2Kings','2Ki','03','027','27',"Then he took his eldest son that should have reigned in his stead, and offered him \emph{for} a burnt offering upon the wall. And there was great indignation against Israel: and they departed from him, and returned to \emph{their} \emph{own} land."]
]

SecondKings04 = [
    ['2Kings','2Ki','04','001','1',"Now there cried a certain woman of the wives of the sons of the prophets unto Elisha, saying, Thy servant my husband is dead; and thou knowest that thy servant did fear the LORD: and the creditor is come to take unto him my two sons to be bondmen."],
    ['2Kings','2Ki','04','002','2',"And Elisha said unto her, What shall I do for thee? tell me, what hast thou in the house? And she said, Thine handmaid hath not any thing in the house, save a pot of oil."],
    ['2Kings','2Ki','04','003','3',"Then he said, Go, borrow thee vessels abroad of all thy neighbours, \emph{even} empty vessels; borrow not a few."],
['2Kings','2Ki','04','004','4',"And when thou art come in, thou shalt shut the door upon thee and upon thy sons, and shalt pour out into all those vessels, and thou shalt set aside that which is full."],
['2Kings','2Ki','04','005','5',"So she went from him, and shut the door upon her and upon her sons, who brought \emph{the} \emph{vessels} to her; and she poured out."],
['2Kings','2Ki','04','006','6',"And it came to pass, when the vessels were full, that she said unto her son, Bring me yet a vessel. And he said unto her, \emph{There} \emph{is} not a vessel more. And the oil stayed."],
['2Kings','2Ki','04','007','7',"Then she came and told the man of God. And he said, Go, sell the oil, and pay thy debt, and live thou and thy children of the rest."],
['2Kings','2Ki','04','008','8',"And it fell on a day, that Elisha passed to Shunem, where \emph{was} a great woman; and she constrained him to eat bread. And \emph{so} it was, \emph{that} as oft as he passed by, he turned in thither to eat bread."],
['2Kings','2Ki','04','009','9',"And she said unto her husband, Behold now, I perceive that this \emph{is} an holy man of God, which passeth by us continually."],
['2Kings','2Ki','04','010','10',"Let us make a little chamber, I pray thee, on the wall; and let us set for him there a bed, and a table, and a stool, and a candlestick: and it shall be, when he cometh to us, that he shall turn in thither."],
['2Kings','2Ki','04','011','11',"And it fell on a day, that he came thither, and he turned into the chamber, and lay there."],
['2Kings','2Ki','04','012','12',"And he said to Gehazi his servant, Call this Shunammite. And when he had called her, she stood before him."],
['2Kings','2Ki','04','013','13',"And he said unto him, Say now unto her, Behold, thou hast been careful for us with all this care; what \emph{is} to be done for thee? wouldest thou be spoken for to the king, or to the captain of the host? And she answered, I dwell among mine own people."],
['2Kings','2Ki','04','014','14',"And he said, What then \emph{is} to be done for her? And Gehazi answered, Verily she hath no child, and her husband is old."],
['2Kings','2Ki','04','015','15',"And he said, Call her. And when he had called her, she stood in the door."],
['2Kings','2Ki','04','016','16',"And he said, About this season, according to the time of life, thou shalt embrace a son. And she said, Nay, my lord, \emph{thou} man of God, do not lie unto thine handmaid."],
['2Kings','2Ki','04','017','17',"And the woman conceived, and bare a son at that season that Elisha had said unto her, according to the time of life."],
['2Kings','2Ki','04','018','18',"And when the child was grown, it fell on a day, that he went out to his father to the reapers."],
['2Kings','2Ki','04','019','19',"And he said unto his father, My head, my head. And he said to a lad, Carry him to his mother."],
['2Kings','2Ki','04','020','20',"And when he had taken him, and brought him to his mother, he sat on her knees till noon, and \emph{then} died."],
['2Kings','2Ki','04','021','21',"And she went up, and laid him on the bed of the man of God, and shut \emph{the} \emph{door} upon him, and went out."],
['2Kings','2Ki','04','022','22',"And she called unto her husband, and said, Send me, I pray thee, one of the young men, and one of the asses, that I may run to the man of God, and come again."],
['2Kings','2Ki','04','023','23',"And he said, Wherefore wilt thou go to him to day? \emph{it} \emph{is} neither new moon, nor sabbath. And she said, \emph{It} \emph{shall} \emph{be} well."],
['2Kings','2Ki','04','024','24',"Then she saddled an ass, and said to her servant, Drive, and go forward; slack not \emph{thy} riding for me, except I bid thee."],
['2Kings','2Ki','04','025','25',"So she went and came unto the man of God to mount Carmel. And it came to pass, when the man of God saw her afar off, that he said to Gehazi his servant, Behold, \emph{yonder} \emph{is} that Shunammite:"],
['2Kings','2Ki','04','026','26',"Run now, I pray thee, to meet her, and say unto her, \emph{Is} \emph{it} well with thee? \emph{is} \emph{it} well with thy husband? \emph{is} \emph{it} well with the child? And she answered, \emph{It} \emph{is} well."],
['2Kings','2Ki','04','027','27',"And when she came to the man of God to the hill, she caught him by the feet: but Gehazi came near to thrust her away. And the man of God said, Let her alone; for her soul \emph{is} vexed within her: and the LORD hath hid \emph{it} from me, and hath not told me."],
['2Kings','2Ki','04','028','28',"Then she said, Did I desire a son of my lord? did I not say, Do not deceive me?"],
['2Kings','2Ki','04','029','29',"Then he said to Gehazi, Gird up thy loins, and take my staff in thine hand, and go thy way: if thou meet any man, salute him not; and if any salute thee, answer him not again: and lay my staff upon the face of the child."],
['2Kings','2Ki','04','030','30',"And the mother of the child said, \emph{As} the LORD liveth, and \emph{as} thy soul liveth, I will not leave thee. And he arose, and followed her."],
['2Kings','2Ki','04','031','31',"And Gehazi passed on before them, and laid the staff upon the face of the child; but \emph{there} \emph{was} neither voice, nor hearing. Wherefore he went again to meet him, and told him, saying, The child is not awaked."],
['2Kings','2Ki','04','032','32',"And when Elisha was come into the house, behold, the child was dead, \emph{and} laid upon his bed."],
['2Kings','2Ki','04','033','33',"He went in therefore, and shut the door upon them twain, and prayed unto the LORD."],
['2Kings','2Ki','04','034','34',"And he went up, and lay upon the child, and put his mouth upon his mouth, and his eyes upon his eyes, and his hands upon his hands: and he stretched himself upon the child; and the flesh of the child waxed warm."],
['2Kings','2Ki','04','035','35',"Then he returned, and walked in the house to and fro; and went up, and stretched himself upon him: and the child sneezed seven times, and the child opened his eyes."],
['2Kings','2Ki','04','036','36',"And he called Gehazi, and said, Call this Shunammite. So he called her. And when she was come in unto him, he said, Take up thy son."],
['2Kings','2Ki','04','037','37',"Then she went in, and fell at his feet, and bowed herself to the ground, and took up her son, and went out."],
['2Kings','2Ki','04','038','38',"And Elisha came again to Gilgal: and \emph{there} \emph{was} a dearth in the land; and the sons of the prophets \emph{were} sitting before him: and he said unto his servant, Set on the great pot, and seethe pottage for the sons of the prophets."],
['2Kings','2Ki','04','039','39',"And one went out into the field to gather herbs, and found a wild vine, and gathered thereof wild gourds his lap full, and came and shred \emph{them} into the pot of pottage: for they knew \emph{them} not."],
['2Kings','2Ki','04','040','40',"So they poured out for the men to eat. And it came to pass, as they were eating of the pottage, that they cried out, and said, O \emph{thou} man of God, \emph{there} \emph{is} death in the pot. And they could not eat \emph{thereof}."],
['2Kings','2Ki','04','041','41',"But he said, Then bring meal. And he cast \emph{it} into the pot; and he said, Pour out for the people, that they may eat. And there was no harm in the pot."],
['2Kings','2Ki','04','042','42',"And there came a man from Baal-shalisha, and brought the man of God bread of the firstfruits, twenty loaves of barley, and full ears of corn in the husk thereof. And he said, Give unto the people, that they may eat."],
['2Kings','2Ki','04','043','43',"And his servitor said, What, should I set this before an hundred men? He said again, Give the people, that they may eat: for thus saith the LORD, They shall eat, and shall leave \emph{thereof}."],
    ['2Kings','2Ki','04','044','44',"So he set \emph{it} before them, and they did eat, and left \emph{thereof}, according to the word of the LORD."]
]

SecondKings05 = [
    ['2Kings','2Ki','05','001','1',"Now Naaman, captain of the host of the king of Syria, was a great man with his master, and honourable, because by him the LORD had given deliverance unto Syria: he was also a mighty man in valour, \emph{but} \emph{he} \emph{was} a leper."],
['2Kings','2Ki','05','002','2',"And the Syrians had gone out by companies, and had brought away captive out of the land of Israel a little maid; and she waited on Naaman's wife."],
['2Kings','2Ki','05','003','3',"And she said unto her mistress, Would God my lord \emph{were} with the prophet that \emph{is} in Samaria! for he would recover him of his leprosy."],
['2Kings','2Ki','05','004','4',"And \emph{one} went in, and told his lord, saying, Thus and thus said the maid that \emph{is} of the land of Israel."],
['2Kings','2Ki','05','005','5',"And the king of Syria said, Go to, go, and I will send a letter unto the king of Israel. And he departed, and took with him ten talents of silver, and six thousand \emph{pieces} of gold, and ten changes of raiment."],
['2Kings','2Ki','05','006','6',"And he brought the letter to the king of Israel, saying, Now when this letter is come unto thee, behold, I have \emph{therewith} sent Naaman my servant to thee, that thou mayest recover him of his leprosy."],
['2Kings','2Ki','05','007','7',"And it came to pass, when the king of Israel had read the letter, that he rent his clothes, and said, \emph{Am} I God, to kill and to make alive, that this man doth send unto me to recover a man of his leprosy? wherefore consider, I pray you, and see how he seeketh a quarrel against me."],
['2Kings','2Ki','05','008','8',"And it was \emph{so}, when Elisha the man of God had heard that the king of Israel had rent his clothes, that he sent to the king, saying, Wherefore hast thou rent thy clothes? let him come now to me, and he shall know that there is a prophet in Israel."],
['2Kings','2Ki','05','009','9',"So Naaman came with his horses and with his chariot, and stood at the door of the house of Elisha."],
['2Kings','2Ki','05','010','10',"And Elisha sent a messenger unto him, saying, Go and wash in Jordan seven times, and thy flesh shall come again to thee, and thou shalt be clean."],
['2Kings','2Ki','05','011','11',"But Naaman was wroth, and went away, and said, Behold, I thought, He will surely come out to me, and stand, and call on the name of the LORD his God, and strike his hand over the place, and recover the leper."],
['2Kings','2Ki','05','012','12',"\emph{Are} not Abana and Pharpar, rivers of Damascus, better than all the waters of Israel? may I not wash in them, and be clean? So he turned and went away in a rage."],
['2Kings','2Ki','05','013','13',"And his servants came near, and spake unto him, and said, My father, \emph{if} the prophet had bid thee \emph{do} \emph{some} great thing, wouldest thou not have done \emph{it}? how much rather then, when he saith to thee, Wash, and be clean?"],
['2Kings','2Ki','05','014','14',"Then went he down, and dipped himself seven times in Jordan, according to the saying of the man of God: and his flesh came again like unto the flesh of a little child, and he was clean."],
['2Kings','2Ki','05','015','15',"And he returned to the man of God, he and all his company, and came, and stood before him: and he said, Behold, now I know that \emph{there} \emph{is} no God in all the earth, but in Israel: now therefore, I pray thee, take a blessing of thy servant."],
['2Kings','2Ki','05','016','16',"But he said, \emph{As} the LORD liveth, before whom I stand, I will receive none. And he urged him to take \emph{it}; but he refused."],
['2Kings','2Ki','05','017','17',"And Naaman said, Shall there not then, I pray thee, be given to thy servant two mules' burden of earth? for thy servant will henceforth offer neither burnt offering nor sacrifice unto other gods, but unto the LORD."],
['2Kings','2Ki','05','018','18',"In this thing the LORD pardon thy servant, \emph{that} when my master goeth into the house of Rimmon to worship there, and he leaneth on my hand, and I bow myself in the house of Rimmon: when I bow down myself in the house of Rimmon, the LORD pardon thy servant in this thing."],
['2Kings','2Ki','05','019','19',"And he said unto him, Go in peace. So he departed from him a little way."],
['2Kings','2Ki','05','020','20',"But Gehazi, the servant of Elisha the man of God, said, Behold, my master hath spared Naaman this Syrian, in not receiving at his hands that which he brought: but, \emph{as} the LORD liveth, I will run after him, and take somewhat of him."],
['2Kings','2Ki','05','021','21',"So Gehazi followed after Naaman. And when Naaman saw \emph{him} running after him, he lighted down from the chariot to meet him, and said, \emph{Is} all well?"],
['2Kings','2Ki','05','022','22',"And he said, All \emph{is} well. My master hath sent me, saying, Behold, even now there be come to me from mount Ephraim two young men of the sons of the prophets: give them, I pray thee, a talent of silver, and two changes of garments."],
['2Kings','2Ki','05','023','23',"And Naaman said, Be content, take two talents. And he urged him, and bound two talents of silver in two bags, with two changes of garments, and laid \emph{them} upon two of his servants; and they bare \emph{them} before him."],
['2Kings','2Ki','05','024','24',"And when he came to the tower, he took \emph{them} from their hand, and bestowed \emph{them} in the house: and he let the men go, and they departed."],
    ['2Kings','2Ki','05','025','25',"But he went in, and stood before his master. And Elisha said unto him, Whence \emph{comest} \emph{thou}, Gehazi? And he said, Thy servant went no whither."],
    ['2Kings','2Ki','05','026','26',"And he said unto him, Went not mine heart \emph{with} \emph{thee}, when the man turned again from his chariot to meet thee? \emph{Is} \emph{it} a time to receive money, and to receive garments, and oliveyards, and vineyards, and sheep, and oxen, and menservants, and maidservants?"],
    ['2Kings','2Ki','05','027','27',"The leprosy therefore of Naaman shall cleave unto thee, and unto thy seed for ever. And he went out from his presence a leper \emph{as} \emph{white} as snow."]
]

SecondKings06 = [
['2Kings','2Ki','06','001','1',"And the sons of the prophets said unto Elisha, Behold now, the place where we dwell with thee is too strait for us."],
['2Kings','2Ki','06','002','2',"Let us go, we pray thee, unto Jordan, and take thence every man a beam, and let us make us a place there, where we may dwell. And he answered, Go ye."],
['2Kings','2Ki','06','003','3',"And one said, Be content, I pray thee, and go with thy servants. And he answered, I will go."],
['2Kings','2Ki','06','004','4',"So he went with them. And when they came to Jordan, they cut down wood."],
['2Kings','2Ki','06','005','5',"But as one was felling a beam, the axe head fell into the water: and he cried, and said, Alas, master! for it was borrowed."],
['2Kings','2Ki','06','006','6',"And the man of God said, Where fell it? And he shewed him the place. And he cut down a stick, and cast \emph{it} in thither; and the iron did swim."],
['2Kings','2Ki','06','007','7',"Therefore said he, Take \emph{it} up to thee. And he put out his hand, and took it."],
['2Kings','2Ki','06','008','8',"Then the king of Syria warred against Israel, and took counsel with his servants, saying, In such and such a place \emph{shall} \emph{be} my camp."],
['2Kings','2Ki','06','009','9',"And the man of God sent unto the king of Israel, saying, Beware that thou pass not such a place; for thither the Syrians are come down."],
['2Kings','2Ki','06','010','10',"And the king of Israel sent to the place which the man of God told him and warned him of, and saved himself there, not once nor twice."],
['2Kings','2Ki','06','011','11',"Therefore the heart of the king of Syria was sore troubled for this thing; and he called his servants, and said unto them, Will ye not shew me which of us \emph{is} for the king of Israel?"],
['2Kings','2Ki','06','012','12',"And one of his servants said, None, my lord, O king: but Elisha, the prophet that \emph{is} in Israel, telleth the king of Israel the words that thou speakest in thy bedchamber."],
['2Kings','2Ki','06','013','13',"And he said, Go and spy where he \emph{is}, that I may send and fetch him. And it was told him, saying, Behold, \emph{he} \emph{is} in Dothan."],
['2Kings','2Ki','06','014','14',"Therefore sent he thither horses, and chariots, and a great host: and they came by night, and compassed the city about."],
['2Kings','2Ki','06','015','15',"And when the servant of the man of God was risen early, and gone forth, behold, an host compassed the city both with horses and chariots. And his servant said unto him, Alas, my master! how shall we do?"],
['2Kings','2Ki','06','016','16',"And he answered, Fear not: for they that \emph{be} with us \emph{are} more than they that \emph{be} with them."],
['2Kings','2Ki','06','017','17',"And Elisha prayed, and said, LORD, I pray thee, open his eyes, that he may see. And the LORD opened the eyes of the young man; and he saw: and, behold, the mountain \emph{was} full of horses and chariots of fire round about Elisha."],
['2Kings','2Ki','06','018','18',"And when they came down to him, Elisha prayed unto the LORD, and said, Smite this people, I pray thee, with blindness. And he smote them with blindness according to the word of Elisha."],
['2Kings','2Ki','06','019','19',"And Elisha said unto them, This \emph{is} not the way, neither \emph{is} this the city: follow me, and I will bring you to the man whom ye seek. But he led them to Samaria."],
['2Kings','2Ki','06','020','20',"And it came to pass, when they were come into Samaria, that Elisha said, LORD, open the eyes of these \emph{men}, that they may see. And the LORD opened their eyes, and they saw; and, behold, \emph{they} \emph{were} in the midst of Samaria."],
['2Kings','2Ki','06','021','21',"And the king of Israel said unto Elisha, when he saw them, My father, shall I smite \emph{them}? shall I smite \emph{them}?"],
['2Kings','2Ki','06','022','22',"And he answered, Thou shalt not smite \emph{them}: wouldest thou smite those whom thou hast taken captive with thy sword and with thy bow? set bread and water before them, that they may eat and drink, and go to their master."],
['2Kings','2Ki','06','023','23',"And he prepared great provision for them: and when they had eaten and drunk, he sent them away, and they went to their master. So the bands of Syria came no more into the land of Israel."],
['2Kings','2Ki','06','024','24',"And it came to pass after this, that Ben-hadad king of Syria gathered all his host, and went up, and besieged Samaria."],
['2Kings','2Ki','06','025','25',"And there was a great famine in Samaria: and, behold, they besieged it, until an ass's head was \emph{sold} for fourscore \emph{pieces} of silver, and the fourth part of a cab of dove's dung for five \emph{pieces} of silver."],
['2Kings','2Ki','06','026','26',"And as the king of Israel was passing by upon the wall, there cried a woman unto him, saying, Help, my lord, O king."],
['2Kings','2Ki','06','027','27',"And he said, If the LORD do not help thee, whence shall I help thee? out of the barnfloor, or out of the winepress?"],
['2Kings','2Ki','06','028','28',"And the king said unto her, What aileth thee? And she answered, This woman said unto me, Give thy son, that we may eat him to day, and we will eat my son to morrow."],
['2Kings','2Ki','06','029','29',"So we boiled my son, and did eat him: and I said unto her on the next day, Give thy son, that we may eat him: and she hath hid her son."],
['2Kings','2Ki','06','030','30',"And it came to pass, when the king heard the words of the woman, that he rent his clothes; and he passed by upon the wall, and the people looked, and, behold, \emph{he} \emph{had} sackcloth within upon his flesh."],
['2Kings','2Ki','06','031','31',"Then he said, God do so and more also to me, if the head of Elisha the son of Shaphat shall stand on him this day."],
['2Kings','2Ki','06','032','32',"But Elisha sat in his house, and the elders sat with him; and \emph{the} \emph{king} sent a man from before him: but ere the messenger came to him, he said to the elders, See ye how this son of a murderer hath sent to take away mine head? look, when the messenger cometh, shut the door, and hold him fast at the door: \emph{is} not the sound of his master's feet behind him?"],
['2Kings','2Ki','06','033','33',"And while he yet talked with them, behold, the messenger came down unto him: and he said, Behold, this evil \emph{is} of the LORD; what should I wait for the LORD any longer?"]
]

SecondKings07 = [
    ['2Kings','2Ki','07','001','1',"Then Elisha said, Hear ye the word of the LORD; Thus saith the LORD, To morrow about this time \emph{shall} a measure of fine flour \emph{be} \emph{sold} for a shekel, and two measures of barley for a shekel, in the gate of Samaria."],
['2Kings','2Ki','07','002','2',"Then a lord on whose hand the king leaned answered the man of God, and said, Behold, \emph{if} the LORD would make windows in heaven, might this thing be? And he said, Behold, thou shalt see \emph{it} with thine eyes, but shalt not eat thereof."],
['2Kings','2Ki','07','003','3',"And there were four leprous men at the entering in of the gate: and they said one to another, Why sit we here until we die?"],
['2Kings','2Ki','07','004','4',"If we say, We will enter into the city, then the famine \emph{is} in the city, and we shall die there: and if we sit still here, we die also. Now therefore come, and let us fall unto the host of the Syrians: if they save us alive, we shall live; and if they kill us, we shall but die."],
['2Kings','2Ki','07','005','5',"And they rose up in the twilight, to go unto the camp of the Syrians: and when they were come to the uttermost part of the camp of Syria, behold, \emph{there} \emph{was} no man there."],
['2Kings','2Ki','07','006','6',"For the Lord had made the host of the Syrians to hear a noise of chariots, and a noise of horses, \emph{even} the noise of a great host: and they said one to another, Lo, the king of Israel hath hired against us the kings of the Hittites, and the kings of the Egyptians, to come upon us."],
['2Kings','2Ki','07','007','7',"Wherefore they arose and fled in the twilight, and left their tents, and their horses, and their asses, even the camp as it \emph{was}, and fled for their life."],
['2Kings','2Ki','07','008','8',"And when these lepers came to the uttermost part of the camp, they went into one tent, and did eat and drink, and carried thence silver, and gold, and raiment, and went and hid \emph{it}; and came again, and entered into another tent, and carried thence \emph{also}, and went and hid \emph{it}."],
['2Kings','2Ki','07','009','9',"Then they said one to another, We do not well: this day \emph{is} a day of good tidings, and we hold our peace: if we tarry till the morning light, some mischief will come upon us: now therefore come, that we may go and tell the king's household."],
['2Kings','2Ki','07','010','10',"So they came and called unto the porter of the city: and they told them, saying, We came to the camp of the Syrians, and, behold, \emph{there} \emph{was} no man there, neither voice of man, but horses tied, and asses tied, and the tents as they \emph{were}."],
['2Kings','2Ki','07','011','11',"And he called the porters; and they told \emph{it} to the king's house within."],
['2Kings','2Ki','07','012','12',"And the king arose in the night, and said unto his servants, I will now shew you what the Syrians have done to us. They know that we \emph{be} hungry; therefore are they gone out of the camp to hide themselves in the field, saying, When they come out of the city, we shall catch them alive, and get into the city."],
['2Kings','2Ki','07','013','13',"And one of his servants answered and said, Let \emph{some} take, I pray thee, five of the horses that remain, which are left in the city, (behold, they \emph{are} as all the multitude of Israel that are left in it: behold, \emph{I} \emph{say}, they \emph{are} even as all the multitude of the Israelites that are consumed:) and let us send and see."],
['2Kings','2Ki','07','014','14',"They took therefore two chariot horses; and the king sent after the host of the Syrians, saying, Go and see."],
['2Kings','2Ki','07','015','15',"And they went after them unto Jordan: and, lo, all the way \emph{was} full of garments and vessels, which the Syrians had cast away in their haste. And the messengers returned, and told the king."],
['2Kings','2Ki','07','016','16',"And the people went out, and spoiled the tents of the Syrians. So a measure of fine flour was \emph{sold} for a shekel, and two measures of barley for a shekel, according to the word of the LORD."],
['2Kings','2Ki','07','017','17',"And the king appointed the lord on whose hand he leaned to have the charge of the gate: and the people trode upon him in the gate, and he died, as the man of God had said, who spake when the king came down to him."],
['2Kings','2Ki','07','018','18',"And it came to pass as the man of God had spoken to the king, saying, Two measures of barley for a shekel, and a measure of fine flour for a shekel, shall be to morrow about this time in the gate of Samaria:"],
['2Kings','2Ki','07','019','19',"And that lord answered the man of God, and said, Now, behold, \emph{if} the LORD should make windows in heaven, might such a thing be? And he said, Behold, thou shalt see it with thine eyes, but shalt not eat thereof."],
['2Kings','2Ki','07','020','20',"And so it fell out unto him: for the people trode upon him in the gate, and he died."]
]

SecondKings08 = [
    ['2Kings','2Ki','08','001','1',"Then spake Elisha unto the woman, whose son he had restored to life, saying, Arise, and go thou and thine household, and sojourn wheresoever thou canst sojourn: for the LORD hath called for a famine; and it shall also come upon the land seven years."],
['2Kings','2Ki','08','002','2',"And the woman arose, and did after the saying of the man of God: and she went with her household, and sojourned in the land of the Philistines seven years."],
['2Kings','2Ki','08','003','3',"And it came to pass at the seven years' end, that the woman returned out of the land of the Philistines: and she went forth to cry unto the king for her house and for her land."],
['2Kings','2Ki','08','004','4',"And the king talked with Gehazi the servant of the man of God, saying, Tell me, I pray thee, all the great things that Elisha hath done."],
['2Kings','2Ki','08','005','5',"And it came to pass, as he was telling the king how he had restored a dead body to life, that, behold, the woman, whose son he had restored to life, cried to the king for her house and for her land. And Gehazi said, My lord, O king, this \emph{is} the woman, and this \emph{is} her son, whom Elisha restored to life."],
['2Kings','2Ki','08','006','6',"And when the king asked the woman, she told him. So the king appointed unto her a certain officer, saying, Restore all that \emph{was} hers, and all the fruits of the field since the day that she left the land, even until now."],
['2Kings','2Ki','08','007','7',"And Elisha came to Damascus; and Ben-hadad the king of Syria was sick; and it was told him, saying, The man of God is come hither."],
['2Kings','2Ki','08','008','8',"And the king said unto Hazael, Take a present in thine hand, and go, meet the man of God, and enquire of the LORD by him, saying, Shall I recover of this disease?"],
['2Kings','2Ki','08','009','9',"So Hazael went to meet him, and took a present with him, even of every good thing of Damascus, forty camels' burden, and came and stood before him, and said, Thy son Ben-hadad king of Syria hath sent me to thee, saying, Shall I recover of this disease?"],
['2Kings','2Ki','08','010','10',"And Elisha said unto him, Go, say unto him, Thou mayest certainly recover: howbeit the LORD hath shewed me that he shall surely die."],
['2Kings','2Ki','08','011','11',"And he settled his countenance stedfastly, until he was ashamed: and the man of God wept."],
['2Kings','2Ki','08','012','12',"And Hazael said, Why weepeth my lord? And he answered, Because I know the evil that thou wilt do unto the children of Israel: their strong holds wilt thou set on fire, and their young men wilt thou slay with the sword, and wilt dash their children, and rip up their women with child."],
['2Kings','2Ki','08','013','13',"And Hazael said, But what, \emph{is} thy servant a dog, that he should do this great thing? And Elisha answered, The LORD hath shewed me that thou \emph{shalt} \emph{be} king over Syria."],
['2Kings','2Ki','08','014','14',"So he departed from Elisha, and came to his master; who said to him, What said Elisha to thee? And he answered, He told me \emph{that} thou shouldest surely recover."],
['2Kings','2Ki','08','015','15',"And it came to pass on the morrow, that he took a thick cloth, and dipped \emph{it} in water, and spread \emph{it} on his face, so that he died: and Hazael reigned in his stead."],
['2Kings','2Ki','08','016','16',"And in the fifth year of Joram the son of Ahab king of Israel, Jehoshaphat \emph{being} then king of Judah, Jehoram the son of Jehoshaphat king of Judah began to reign."],
['2Kings','2Ki','08','017','17',"Thirty and two years old was he when he began to reign; and he reigned eight years in Jerusalem."],
['2Kings','2Ki','08','018','18',"And he walked in the way of the kings of Israel, as did the house of Ahab: for the daughter of Ahab was his wife: and he did evil in the sight of the LORD."],
['2Kings','2Ki','08','019','19',"Yet the LORD would not destroy Judah for David his servant's sake, as he promised him to give him alway a light, \emph{and} to his children."],
['2Kings','2Ki','08','020','20',"In his days Edom revolted from under the hand of Judah, and made a king over themselves."],
['2Kings','2Ki','08','021','21',"So Joram went over to Zair, and all the chariots with him: and he rose by night, and smote the Edomites which compassed him about, and the captains of the chariots: and the people fled into their tents."],
['2Kings','2Ki','08','022','22',"Yet Edom revolted from under the hand of Judah unto this day. Then Libnah revolted at the same time."],
['2Kings','2Ki','08','023','23',"And the rest of the acts of Joram, and all that he did, \emph{are} they not written in the book of the chronicles of the kings of Judah?"],
['2Kings','2Ki','08','024','24',"And Joram slept with his fathers, and was buried with his fathers in the city of David: and Ahaziah his son reigned in his stead."],
['2Kings','2Ki','08','025','25',"In the twelfth year of Joram the son of Ahab king of Israel did Ahaziah the son of Jehoram king of Judah begin to reign."],
['2Kings','2Ki','08','026','26',"Two and twenty years old \emph{was} Ahaziah when he began to reign; and he reigned one year in Jerusalem. And his mother's name \emph{was} Athaliah, the daughter of Omri king of Israel."],
['2Kings','2Ki','08','027','27',"And he walked in the way of the house of Ahab, and did evil in the sight of the LORD, as \emph{did} the house of Ahab: for he \emph{was} the son in law of the house of Ahab."],
['2Kings','2Ki','08','028','28',"And he went with Joram the son of Ahab to the war against Hazael king of Syria in Ramoth-gilead; and the Syrians wounded Joram."],
['2Kings','2Ki','08','029','29',"And king Joram went back to be healed in Jezreel of the wounds which the Syrians had given him at Ramah, when he fought against Hazael king of Syria. And Ahaziah the son of Jehoram king of Judah went down to see Joram the son of Ahab in Jezreel, because he was sick."]
]

SecondKings09 = [
    ['2Kings','2Ki','09','001','1',"And Elisha the prophet called one of the children of the prophets, and said unto him, Gird up thy loins, and take this box of oil in thine hand, and go to Ramoth-gilead:"],
['2Kings','2Ki','09','002','2',"And when thou comest thither, look out there Jehu the son of Jehoshaphat the son of Nimshi, and go in, and make him arise up from among his brethren, and carry him to an inner chamber;"],
['2Kings','2Ki','09','003','3',"Then take the box of oil, and pour \emph{it} on his head, and say, Thus saith the LORD, I have anointed thee king over Israel. Then open the door, and flee, and tarry not."],
['2Kings','2Ki','09','004','4',"So the young man, \emph{even} the young man the prophet, went to Ramoth-gilead."],
['2Kings','2Ki','09','005','5',"And when he came, behold, the captains of the host \emph{were} sitting; and he said, I have an errand to thee, O captain. And Jehu said, Unto which of all us? And he said, To thee, O captain."],
['2Kings','2Ki','09','006','6',"And he arose, and went into the house; and he poured the oil on his head, and said unto him, Thus saith the LORD God of Israel, I have anointed thee king over the people of the LORD, \emph{even} over Israel."],
['2Kings','2Ki','09','007','7',"And thou shalt smite the house of Ahab thy master, that I may avenge the blood of my servants the prophets, and the blood of all the servants of the LORD, at the hand of Jezebel."],
['2Kings','2Ki','09','008','8',"For the whole house of Ahab shall perish: and I will cut off from Ahab him that pisseth against the wall, and him that is shut up and left in Israel:"],
['2Kings','2Ki','09','009','9',"And I will make the house of Ahab like the house of Jeroboam the son of Nebat, and like the house of Baasha the son of Ahijah:"],
['2Kings','2Ki','09','010','10',"And the dogs shall eat Jezebel in the portion of Jezreel, and \emph{there} \emph{shall} \emph{be} none to bury \emph{her}. And he opened the door, and fled."],
['2Kings','2Ki','09','011','11',"Then Jehu came forth to the servants of his lord: and \emph{one} said unto him, \emph{Is} all well? wherefore came this mad \emph{fellow} to thee? And he said unto them, Ye know the man, and his communication."],
['2Kings','2Ki','09','012','12',"And they said, \emph{It} \emph{is} false; tell us now. And he said, Thus and thus spake he to me, saying, Thus saith the LORD, I have anointed thee king over Israel."],
['2Kings','2Ki','09','013','13',"Then they hasted, and took every man his garment, and put \emph{it} under him on the top of the stairs, and blew with trumpets, saying, Jehu is king."],
['2Kings','2Ki','09','014','14',"So Jehu the son of Jehoshaphat the son of Nimshi conspired against Joram. (Now Joram had kept Ramoth-gilead, he and all Israel, because of Hazael king of Syria."],
['2Kings','2Ki','09','015','15',"But king Joram was returned to be healed in Jezreel of the wounds which the Syrians had given him, when he fought with Hazael king of Syria.) And Jehu said, If it be your minds, \emph{then} let none go forth \emph{nor} escape out of the city to go to tell \emph{it} in Jezreel."],
['2Kings','2Ki','09','016','16',"So Jehu rode in a chariot, and went to Jezreel; for Joram lay there. And Ahaziah king of Judah was come down to see Joram."],
['2Kings','2Ki','09','017','17',"And there stood a watchman on the tower in Jezreel, and he spied the company of Jehu as he came, and said, I see a company. And Joram said, Take an horseman, and send to meet them, and let him say, \emph{Is} \emph{it} peace?"],
['2Kings','2Ki','09','018','18',"So there went one on horseback to meet him, and said, Thus saith the king, \emph{Is} \emph{it} peace? And Jehu said, What hast thou to do with peace? turn thee behind me. And the watchman told, saying, The messenger came to them, but he cometh not again."],
['2Kings','2Ki','09','019','19',"Then he sent out a second on horseback, which came to them, and said, Thus saith the king, \emph{Is} \emph{it} peace? And Jehu answered, What hast thou to do with peace? turn thee behind me."],
['2Kings','2Ki','09','020','20',"And the watchman told, saying, He came even unto them, and cometh not again: and the driving \emph{is} like the driving of Jehu the son of Nimshi; for he driveth furiously."],
['2Kings','2Ki','09','021','21',"And Joram said, Make ready. And his chariot was made ready. And Joram king of Israel and Ahaziah king of Judah went out, each in his chariot, and they went out against Jehu, and met him in the portion of Naboth the Jezreelite."],
['2Kings','2Ki','09','022','22',"And it came to pass, when Joram saw Jehu, that he said, \emph{Is} \emph{it} peace, Jehu? And he answered, What peace, so long as the whoredoms of thy mother Jezebel and her witchcrafts \emph{are} \emph{so} many?"],
['2Kings','2Ki','09','023','23',"And Joram turned his hands, and fled, and said to Ahaziah, \emph{There} \emph{is} treachery, O Ahaziah."],
['2Kings','2Ki','09','024','24',"And Jehu drew a bow with his full strength, and smote Jehoram between his arms, and the arrow went out at his heart, and he sunk down in his chariot."],
['2Kings','2Ki','09','025','25',"Then said \emph{Jehu} to Bidkar his captain, Take up, \emph{and} cast him in the portion of the field of Naboth the Jezreelite: for remember how that, when I and thou rode together after Ahab his father, the LORD laid this burden upon him;"],
['2Kings','2Ki','09','026','26',"Surely I have seen yesterday the blood of Naboth, and the blood of his sons, saith the LORD; and I will requite thee in this plat, saith the LORD. Now therefore take \emph{and} cast him into the plat \emph{of} \emph{ground}, according to the word of the LORD."],
['2Kings','2Ki','09','027','27',"But when Ahaziah the king of Judah saw \emph{this}, he fled by the way of the garden house. And Jehu followed after him, and said, Smite him also in the chariot. \emph{And} \emph{they} \emph{did} \emph{so} at the going up to Gur, which \emph{is} by Ibleam. And he fled to Megiddo, and died there."],
['2Kings','2Ki','09','028','28',"And his servants carried him in a chariot to Jerusalem, and buried him in his sepulchre with his fathers in the city of David."],
['2Kings','2Ki','09','029','29',"And in the eleventh year of Joram the son of Ahab began Ahaziah to reign over Judah."],
['2Kings','2Ki','09','030','30',"And when Jehu was come to Jezreel, Jezebel heard \emph{of} \emph{it}; and she painted her face, and tired her head, and looked out at a window."],
['2Kings','2Ki','09','031','31',"And as Jehu entered in at the gate, she said, \emph{Had} Zimri peace, who slew his master?"],
['2Kings','2Ki','09','032','32',"And he lifted up his face to the window, and said, Who \emph{is} on my side? who? And there looked out to him two \emph{or} three eunuchs."],
['2Kings','2Ki','09','033','33',"And he said, Throw her down. So they threw her down: and \emph{some} of her blood was sprinkled on the wall, and on the horses: and he trode her under foot."],
['2Kings','2Ki','09','034','34',"And when he was come in, he did eat and drink, and said, Go, see now this cursed \emph{woman}, and bury her: for she \emph{is} a king's daughter."],
['2Kings','2Ki','09','035','35',"And they went to bury her: but they found no more of her than the skull, and the feet, and the palms of \emph{her} hands."],
['2Kings','2Ki','09','036','36',"Wherefore they came again, and told him. And he said, This \emph{is} the word of the LORD, which he spake by his servant Elijah the Tishbite, saying, In the portion of Jezreel shall dogs eat the flesh of Jezebel:"],
['2Kings','2Ki','09','037','37',"And the carcase of Jezebel shall be as dung upon the face of the field in the portion of Jezreel; \emph{so} that they shall not say, This \emph{is} Jezebel."]
]

SecondKings10 = [
    ['2Kings','2Ki','10','001','1',"And Ahab had seventy sons in Samaria. And Jehu wrote letters, and sent to Samaria, unto the rulers of Jezreel, to the elders, and to them that brought up Ahab's \emph{children}, saying,"],
    ['2Kings','2Ki','10','002','2',"Now as soon as this letter cometh to you, seeing your master's sons \emph{are} with you, and \emph{there} \emph{are} with you chariots and horses, a fenced city also, and armour;"],
    ['2Kings','2Ki','10','003','3',"Look even out the best and meetest of your master's sons, and set \emph{him} on his father's throne, and fight for your master's house."],
['2Kings','2Ki','10','004','4',"But they were exceedingly afraid, and said, Behold, two kings stood not before him: how then shall we stand?"],
['2Kings','2Ki','10','005','5',"And he that \emph{was} over the house, and he that \emph{was} over the city, the elders also, and the bringers up \emph{of} \emph{the} \emph{children}, sent to Jehu, saying, We \emph{are} thy servants, and will do all that thou shalt bid us; we will not make any king: do thou \emph{that} \emph{which} \emph{is} good in thine eyes."],
['2Kings','2Ki','10','006','6',"Then he wrote a letter the second time to them, saying, If ye \emph{be} mine, and \emph{if} ye will hearken unto my voice, take ye the heads of the men your master's sons, and come to me to Jezreel by to morrow this time. Now the king's sons, \emph{being} seventy persons, \emph{were} with the great men of the city, which brought them up."],
['2Kings','2Ki','10','007','7',"And it came to pass, when the letter came to them, that they took the king's sons, and slew seventy persons, and put their heads in baskets, and sent him \emph{them} to Jezreel."],
['2Kings','2Ki','10','008','8',"And there came a messenger, and told him, saying, They have brought the heads of the king's sons. And he said, Lay ye them in two heaps at the entering in of the gate until the morning."],
['2Kings','2Ki','10','009','9',"And it came to pass in the morning, that he went out, and stood, and said to all the people, Ye \emph{be} righteous: behold, I conspired against my master, and slew him: but who slew all these?"],
['2Kings','2Ki','10','010','10',"Know now that there shall fall unto the earth nothing of the word of the LORD, which the LORD spake concerning the house of Ahab: for the LORD hath done \emph{that} which he spake by his servant Elijah."],
['2Kings','2Ki','10','011','11',"So Jehu slew all that remained of the house of Ahab in Jezreel, and all his great men, and his kinsfolks, and his priests, until he left him none remaining."],
['2Kings','2Ki','10','012','12',"And he arose and departed, and came to Samaria. \emph{And} as he \emph{was} at the shearing house in the way,"],
['2Kings','2Ki','10','013','13',"Jehu met with the brethren of Ahaziah king of Judah, and said, Who \emph{are} ye? And they answered, We \emph{are} the brethren of Ahaziah; and we go down to salute the children of the king and the children of the queen."],
['2Kings','2Ki','10','014','14',"And he said, Take them alive. And they took them alive, and slew them at the pit of the shearing house, \emph{even} two and forty men; neither left he any of them."],
['2Kings','2Ki','10','015','15',"And when he was departed thence, he lighted on Jehonadab the son of Rechab \emph{coming} to meet him: and he saluted him, and said to him, Is thine heart right, as my heart \emph{is} with thy heart? And Jehonadab answered, It is. If it be, give \emph{me} thine hand. And he gave \emph{him} his hand; and he took him up to him into the chariot."],
['2Kings','2Ki','10','016','16',"And he said, Come with me, and see my zeal for the LORD. So they made him ride in his chariot."],
['2Kings','2Ki','10','017','17',"And when he came to Samaria, he slew all that remained unto Ahab in Samaria, till he had destroyed him, according to the saying of the LORD, which he spake to Elijah."],
['2Kings','2Ki','10','018','18',"And Jehu gathered all the people together, and said unto them, Ahab served Baal a little; \emph{but} Jehu shall serve him much."],
['2Kings','2Ki','10','019','19',"Now therefore call unto me all the prophets of Baal, all his servants, and all his priests; let none be wanting: for I have a great sacrifice \emph{to} \emph{do} to Baal; whosoever shall be wanting, he shall not live. But Jehu did \emph{it} in subtilty, to the intent that he might destroy the worshippers of Baal."],
['2Kings','2Ki','10','020','20',"And Jehu said, Proclaim a solemn assembly for Baal. And they proclaimed \emph{it}."],
['2Kings','2Ki','10','021','21',"And Jehu sent through all Israel: and all the worshippers of Baal came, so that there was not a man left that came not. And they came into the house of Baal; and the house of Baal was full from one end to another."],
['2Kings','2Ki','10','022','22',"And he said unto him that \emph{was} over the vestry, Bring forth vestments for all the worshippers of Baal. And he brought them forth vestments."],
['2Kings','2Ki','10','023','23',"And Jehu went, and Jehonadab the son of Rechab, into the house of Baal, and said unto the worshippers of Baal, Search, and look that there be here with you none of the servants of the LORD, but the worshippers of Baal only."],
['2Kings','2Ki','10','024','24',"And when they went in to offer sacrifices and burnt offerings, Jehu appointed fourscore men without, and said, \emph{If} any of the men whom I have brought into your hands escape, \emph{he} \emph{that} \emph{letteth} \emph{him} \emph{go}, his life \emph{shall} \emph{be} for the life of him."],
['2Kings','2Ki','10','025','25',"And it came to pass, as soon as he had made an end of offering the burnt offering, that Jehu said to the guard and to the captains, Go in, \emph{and} slay them; let none come forth. And they smote them with the edge of the sword; and the guard and the captains cast \emph{them} out, and went to the city of the house of Baal."],
['2Kings','2Ki','10','026','26',"And they brought forth the images out of the house of Baal, and burned them."],
['2Kings','2Ki','10','027','27',"And they brake down the image of Baal, and brake down the house of Baal, and made it a draught house unto this day."],
['2Kings','2Ki','10','028','28',"Thus Jehu destroyed Baal out of Israel."],
['2Kings','2Ki','10','029','29',"Howbeit \emph{from} the sins of Jeroboam the son of Nebat, who made Israel to sin, Jehu departed not from after them, \emph{to} \emph{wit}, the golden calves that \emph{were} in Beth-el, and that \emph{were} in Dan."],
['2Kings','2Ki','10','030','30',"And the LORD said unto Jehu, Because thou hast done well in executing \emph{that} \emph{which} \emph{is} right in mine eyes, \emph{and} hast done unto the house of Ahab according to all that \emph{was} in mine heart, thy children of the fourth \emph{generation} shall sit on the throne of Israel."],
['2Kings','2Ki','10','031','31',"But Jehu took no heed to walk in the law of the LORD God of Israel with all his heart: for he departed not from the sins of Jeroboam, which made Israel to sin."],
['2Kings','2Ki','10','032','32',"In those days the LORD began to cut Israel short: and Hazael smote them in all the coasts of Israel;"],
['2Kings','2Ki','10','033','33',"From Jordan eastward, all the land of Gilead, the Gadites, and the Reubenites, and the Manassites, from Aroer, which \emph{is} by the river Arnon, even Gilead and Bashan."],
['2Kings','2Ki','10','034','34',"Now the rest of the acts of Jehu, and all that he did, and all his might, \emph{are} they not written in the book of the chronicles of the kings of Israel?"],
['2Kings','2Ki','10','035','35',"And Jehu slept with his fathers: and they buried him in Samaria. And Jehoahaz his son reigned in his stead."],
['2Kings','2Ki','10','036','36',"And the time that Jehu reigned over Israel in Samaria \emph{was} twenty and eight years."]
]

SecondKings11 = [
    ['2Kings','2Ki','11','001','1',"And when Athaliah the mother of Ahaziah saw that her son was dead, she arose and destroyed all the seed royal."],
['2Kings','2Ki','11','002','2',"But Jehosheba, the daughter of king Joram, sister of Ahaziah, took Joash the son of Ahaziah, and stole him from among the king's sons \emph{which} \emph{were} slain; and they hid him, \emph{even} him and his nurse, in the bedchamber from Athaliah, so that he was not slain."],
['2Kings','2Ki','11','003','3',"And he was with her hid in the house of the LORD six years. And Athaliah did reign over the land."],
['2Kings','2Ki','11','004','4',"And the seventh year Jehoiada sent and fetched the rulers over hundreds, with the captains and the guard, and brought them to him into the house of the LORD, and made a covenant with them, and took an oath of them in the house of the LORD, and shewed them the king's son."],
['2Kings','2Ki','11','005','5',"And he commanded them, saying, This \emph{is} the thing that ye shall do; A third part of you that enter in on the sabbath shall even be keepers of the watch of the king's house;"],
['2Kings','2Ki','11','006','6',"And a third part \emph{shall} \emph{be} at the gate of Sur; and a third part at the gate behind the guard: so shall ye keep the watch of the house, that it be not broken down."],
['2Kings','2Ki','11','007','7',"And two parts of all you that go forth on the sabbath, even they shall keep the watch of the house of the LORD about the king."],
['2Kings','2Ki','11','008','8',"And ye shall compass the king round about, every man with his weapons in his hand: and he that cometh within the ranges, let him be slain: and be ye with the king as he goeth out and as he cometh in."],
['2Kings','2Ki','11','009','9',"And the captains over the hundreds did according to all \emph{things} that Jehoiada the priest commanded: and they took every man his men that were to come in on the sabbath, with them that should go out on the sabbath, and came to Jehoiada the priest."],
['2Kings','2Ki','11','010','10',"And to the captains over hundreds did the priest give king David's spears and shields, that \emph{were} in the temple of the LORD."],
['2Kings','2Ki','11','011','11',"And the guard stood, every man with his weapons in his hand, round about the king, from the right corner of the temple to the left corner of the temple, \emph{along} by the altar and the temple."],
['2Kings','2Ki','11','012','12',"And he brought forth the king's son, and put the crown upon him, and \emph{gave} \emph{him} the testimony; and they made him king, and anointed him; and they clapped their hands, and said, God save the king."],
['2Kings','2Ki','11','013','13',"And when Athaliah heard the noise of the guard \emph{and} of the people, she came to the people into the temple of the LORD."],
['2Kings','2Ki','11','014','14',"And when she looked, behold, the king stood by a pillar, as the manner \emph{was}, and the princes and the trumpeters by the king, and all the people of the land rejoiced, and blew with trumpets: and Athaliah rent her clothes, and cried, Treason, Treason."],
['2Kings','2Ki','11','015','15',"But Jehoiada the priest commanded the captains of the hundreds, the officers of the host, and said unto them, Have her forth without the ranges: and him that followeth her kill with the sword. For the priest had said, Let her not be slain in the house of the LORD."],
['2Kings','2Ki','11','016','16',"And they laid hands on her; and she went by the way by the which the horses came into the king's house: and there was she slain."],
['2Kings','2Ki','11','017','17',"And Jehoiada made a covenant between the LORD and the king and the people, that they should be the LORD'S people; between the king also and the people."],
['2Kings','2Ki','11','018','18',"And all the people of the land went into the house of Baal, and brake it down; his altars and his images brake they in pieces thoroughly, and slew Mattan the priest of Baal before the altars. And the priest appointed officers over the house of the LORD."],
['2Kings','2Ki','11','019','19',"And he took the rulers over hundreds, and the captains, and the guard, and all the people of the land; and they brought down the king from the house of the LORD, and came by the way of the gate of the guard to the king's house. And he sat on the throne of the kings."],
['2Kings','2Ki','11','020','20',"And all the people of the land rejoiced, and the city was in quiet: and they slew Athaliah with the sword \emph{beside} the king's house."],
['2Kings','2Ki','11','021','21',"Seven years old \emph{was} Jehoash when he began to reign."]
]

SecondKings12 = [
    ['2Kings','2Ki','12','001','1',"In the seventh year of Jehu Jehoash began to reign; and forty years reigned he in Jerusalem. And his mother's name \emph{was} Zibiah of Beer-sheba."],
['2Kings','2Ki','12','002','2',"And Jehoash did \emph{that} \emph{which} \emph{was} right in the sight of the LORD all his days wherein Jehoiada the priest instructed him."],
['2Kings','2Ki','12','003','3',"But the high places were not taken away: the people still sacrificed and burnt incense in the high places."],
['2Kings','2Ki','12','004','4',"And Jehoash said to the priests, All the money of the dedicated things that is brought into the house of the LORD, \emph{even} the money of every one that passeth \emph{the} \emph{account}, the money that every man is set at, \emph{and} all the money that cometh into any man's heart to bring into the house of the LORD,"],
['2Kings','2Ki','12','005','5',"Let the priests take \emph{it} to them, every man of his acquaintance: and let them repair the breaches of the house, wheresoever any breach shall be found."],
['2Kings','2Ki','12','006','6',"But it was \emph{so,} \emph{that} in the three and twentieth year of king Jehoash the priests had not repaired the breaches of the house."],
['2Kings','2Ki','12','007','7',"Then king Jehoash called for Jehoiada the priest, and the \emph{other} priests, and said unto them, Why repair ye not the breaches of the house? now therefore receive no \emph{more} money of your acquaintance, but deliver it for the breaches of the house."],
['2Kings','2Ki','12','008','8',"And the priests consented to receive no \emph{more} money of the people, neither to repair the breaches of the house."],
['2Kings','2Ki','12','009','9',"But Jehoiada the priest took a chest, and bored a hole in the lid of it, and set it beside the altar, on the right side as one cometh into the house of the LORD: and the priests that kept the door put therein all the money \emph{that} \emph{was} brought into the house of the LORD."],
['2Kings','2Ki','12','010','10',"And it was \emph{so}, when they saw that \emph{there} \emph{was} much money in the chest, that the king's scribe and the high priest came up, and they put up in bags, and told the money that was found in the house of the LORD."],
['2Kings','2Ki','12','011','11',"And they gave the money, being told, into the hands of them that did the work, that had the oversight of the house of the LORD: and they laid it out to the carpenters and builders, that wrought upon the house of the LORD,"],
['2Kings','2Ki','12','012','12',"And to masons, and hewers of stone, and to buy timber and hewed stone to repair the breaches of the house of the LORD, and for all that was laid out for the house to repair \emph{it}."],
['2Kings','2Ki','12','013','13',"Howbeit there were not made for the house of the LORD bowls of silver, snuffers, basons, trumpets, any vessels of gold, or vessels of silver, of the money \emph{that} \emph{was} brought into the house of the LORD:"],
['2Kings','2Ki','12','014','14',"But they gave that to the workmen, and repaired therewith the house of the LORD."],
['2Kings','2Ki','12','015','15',"Moreover they reckoned not with the men, into whose hand they delivered the money to be bestowed on workmen: for they dealt faithfully."],
['2Kings','2Ki','12','016','16',"The trespass money and sin money was not brought into the house of the LORD: it was the priests'."],
['2Kings','2Ki','12','017','17',"Then Hazael king of Syria went up, and fought against Gath, and took it: and Hazael set his face to go up to Jerusalem."],
['2Kings','2Ki','12','018','18',"And Jehoash king of Judah took all the hallowed things that Jehoshaphat, and Jehoram, and Ahaziah, his fathers, kings of Judah, had dedicated, and his own hallowed things, and all the gold \emph{that} \emph{was} found in the treasures of the house of the LORD, and in the king's house, and sent \emph{it} to Hazael king of Syria: and he went away from Jerusalem."],
['2Kings','2Ki','12','019','19',"And the rest of the acts of Joash, and all that he did, \emph{are} they not written in the book of the chronicles of the kings of Judah?"],
['2Kings','2Ki','12','020','20',"And his servants arose, and made a conspiracy, and slew Joash in the house of Millo, which goeth down to Silla."],
['2Kings','2Ki','12','021','21',"For Jozachar the son of Shimeath, and Jehozabad the son of Shomer, his servants, smote him, and he died; and they buried him with his fathers in the city of David: and Amaziah his son reigned in his stead."]
]

SecondKings13 = [
    ['2Kings','2Ki','13','001','1',"In the three and twentieth year of Joash the son of Ahaziah king of Judah Jehoahaz the son of Jehu began to reign over Israel in Samaria, \emph{and} \emph{reigned} seventeen years."],
['2Kings','2Ki','13','002','2',"And he did \emph{that} \emph{which} \emph{was} evil in the sight of the LORD, and followed the sins of Jeroboam the son of Nebat, which made Israel to sin; he departed not therefrom."],
['2Kings','2Ki','13','003','3',"And the anger of the LORD was kindled against Israel, and he delivered them into the hand of Hazael king of Syria, and into the hand of Ben-hadad the son of Hazael, all \emph{their} days."],
['2Kings','2Ki','13','004','4',"And Jehoahaz besought the LORD, and the LORD hearkened unto him: for he saw the oppression of Israel, because the king of Syria oppressed them."],
['2Kings','2Ki','13','005','5',"(And the LORD gave Israel a saviour, so that they went out from under the hand of the Syrians: and the children of Israel dwelt in their tents, as beforetime."],
['2Kings','2Ki','13','006','6',"Nevertheless they departed not from the sins of the house of Jeroboam, who made Israel sin, \emph{but} walked therein: and there remained the grove also in Samaria.)"],
['2Kings','2Ki','13','007','7',"Neither did he leave of the people to Jehoahaz but fifty horsemen, and ten chariots, and ten thousand footmen; for the king of Syria had destroyed them, and had made them like the dust by threshing."],
['2Kings','2Ki','13','008','8',"Now the rest of the acts of Jehoahaz, and all that he did, and his might, \emph{are} they not written in the book of the chronicles of the kings of Israel?"],
['2Kings','2Ki','13','009','9',"And Jehoahaz slept with his fathers; and they buried him in Samaria: and Joash his son reigned in his stead."],
['2Kings','2Ki','13','010','10',"In the thirty and seventh year of Joash king of Judah began Jehoash the son of Jehoahaz to reign over Israel in Samaria, \emph{and} \emph{reigned} sixteen years."],
['2Kings','2Ki','13','011','11',"And he did \emph{that} \emph{which} \emph{was} evil in the sight of the LORD; he departed not from all the sins of Jeroboam the son of Nebat, who made Israel sin: \emph{but} he walked therein."],
['2Kings','2Ki','13','012','12',"And the rest of the acts of Joash, and all that he did, and his might wherewith he fought against Amaziah king of Judah, \emph{are} they not written in the book of the chronicles of the kings of Israel?"],
['2Kings','2Ki','13','013','13',"And Joash slept with his fathers; and Jeroboam sat upon his throne: and Joash was buried in Samaria with the kings of Israel."],
['2Kings','2Ki','13','014','14',"Now Elisha was fallen sick of his sickness whereof he died. And Joash the king of Israel came down unto him, and wept over his face, and said, O my father, my father, the chariot of Israel, and the horsemen thereof."],
['2Kings','2Ki','13','015','15',"And Elisha said unto him, Take bow and arrows. And he took unto him bow and arrows."],
['2Kings','2Ki','13','016','16',"And he said to the king of Israel, Put thine hand upon the bow. And he put his hand \emph{upon} \emph{it}: and Elisha put his hands upon the king's hands."],
['2Kings','2Ki','13','017','17',"And he said, Open the window eastward. And he opened \emph{it}. Then Elisha said, Shoot. And he shot. And he said, The arrow of the LORD'S deliverance, and the arrow of deliverance from Syria: for thou shalt smite the Syrians in Aphek, till thou have consumed \emph{them}."],
['2Kings','2Ki','13','018','18',"And he said, Take the arrows. And he took \emph{them}. And he said unto the king of Israel, Smite upon the ground. And he smote thrice, and stayed."],
['2Kings','2Ki','13','019','19',"And the man of God was wroth with him, and said, Thou shouldest have smitten five or six times; then hadst thou smitten Syria till thou hadst consumed \emph{it}: whereas now thou shalt smite Syria \emph{but} thrice."],
['2Kings','2Ki','13','020','20',"And Elisha died, and they buried him. And the bands of the Moabites invaded the land at the coming in of the year."],
['2Kings','2Ki','13','021','21',"And it came to pass, as they were burying a man, that, behold, they spied a band \emph{of} \emph{men}; and they cast the man into the sepulchre of Elisha: and when the man was let down, and touched the bones of Elisha, he revived, and stood up on his feet."],
['2Kings','2Ki','13','022','22',"But Hazael king of Syria oppressed Israel all the days of Jehoahaz."],
['2Kings','2Ki','13','023','23',"And the LORD was gracious unto them, and had compassion on them, and had respect unto them, because of his covenant with Abraham, Isaac, and Jacob, and would not destroy them, neither cast he them from his presence as yet."],
['2Kings','2Ki','13','024','24',"So Hazael king of Syria died; and Ben-hadad his son reigned in his stead."],
['2Kings','2Ki','13','025','25',"And Jehoash the son of Jehoahaz took again out of the hand of Ben-hadad the son of Hazael the cities, which he had taken out of the hand of Jehoahaz his father by war. Three times did Joash beat him, and recovered the cities of Israel."]
]

SecondKings14 = [
    ['2Kings','2Ki','14','001','1',"In the second year of Joash son of Jehoahaz king of Israel reigned Amaziah the son of Joash king of Judah."],
['2Kings','2Ki','14','002','2',"He was twenty and five years old when he began to reign, and reigned twenty and nine years in Jerusalem. And his mother's name \emph{was} Jehoaddan of Jerusalem."],
['2Kings','2Ki','14','003','3',"And he did \emph{that} \emph{which} \emph{was} right in the sight of the LORD, yet not like David his father: he did according to all things as Joash his father did."],
['2Kings','2Ki','14','004','4',"Howbeit the high places were not taken away: as yet the people did sacrifice and burnt incense on the high places."],
['2Kings','2Ki','14','005','5',"And it came to pass, as soon as the kingdom was confirmed in his hand, that he slew his servants which had slain the king his father."],
['2Kings','2Ki','14','006','6',"But the children of the murderers he slew not: according unto that which is written in the book of the law of Moses, wherein the LORD commanded, saying, The fathers shall not be put to death for the children, nor the children be put to death for the fathers; but every man shall be put to death for his own sin."],
['2Kings','2Ki','14','007','7',"He slew of Edom in the valley of salt ten thousand, and took Selah by war, and called the name of it Joktheel unto this day."],
['2Kings','2Ki','14','008','8',"Then Amaziah sent messengers to Jehoash, the son of Jehoahaz son of Jehu, king of Israel, saying, Come, let us look one another in the face."],
['2Kings','2Ki','14','009','9',"And Jehoash the king of Israel sent to Amaziah king of Judah, saying, The thistle that \emph{was} in Lebanon sent to the cedar that \emph{was} in Lebanon, saying, Give thy daughter to my son to wife: and there passed by a wild beast that \emph{was} in Lebanon, and trode down the thistle."],
['2Kings','2Ki','14','010','10',"Thou hast indeed smitten Edom, and thine heart hath lifted thee up: glory \emph{of} \emph{this}, and tarry at home: for why shouldest thou meddle to \emph{thy} hurt, that thou shouldest fall, \emph{even} thou, and Judah with thee?"],
['2Kings','2Ki','14','011','11',"But Amaziah would not hear. Therefore Jehoash king of Israel went up; and he and Amaziah king of Judah looked one another in the face at Beth-shemesh, which \emph{belongeth} to Judah."],
['2Kings','2Ki','14','012','12',"And Judah was put to the worse before Israel; and they fled every man to their tents."],
['2Kings','2Ki','14','013','13',"And Jehoash king of Israel took Amaziah king of Judah, the son of Jehoash the son of Ahaziah, at Beth-shemesh, and came to Jerusalem, and brake down the wall of Jerusalem from the gate of Ephraim unto the corner gate, four hundred cubits."],
['2Kings','2Ki','14','014','14',"And he took all the gold and silver, and all the vessels that were found in the house of the LORD, and in the treasures of the king's house, and hostages, and returned to Samaria."],
['2Kings','2Ki','14','015','15',"Now the rest of the acts of Jehoash which he did, and his might, and how he fought with Amaziah king of Judah, \emph{are} they not written in the book of the chronicles of the kings of Israel?"],
['2Kings','2Ki','14','016','16',"And Jehoash slept with his fathers, and was buried in Samaria with the kings of Israel; and Jeroboam his son reigned in his stead."],
['2Kings','2Ki','14','017','17',"And Amaziah the son of Joash king of Judah lived after the death of Jehoash son of Jehoahaz king of Israel fifteen years."],
['2Kings','2Ki','14','018','18',"And the rest of the acts of Amaziah, \emph{are} they not written in the book of the chronicles of the kings of Judah?"],
['2Kings','2Ki','14','019','19',"Now they made a conspiracy against him in Jerusalem: and he fled to Lachish; but they sent after him to Lachish, and slew him there."],
['2Kings','2Ki','14','020','20',"And they brought him on horses: and he was buried at Jerusalem with his fathers in the city of David."],
['2Kings','2Ki','14','021','21',"And all the people of Judah took Azariah, which \emph{was} sixteen years old, and made him king instead of his father Amaziah."],
['2Kings','2Ki','14','022','22',"He built Elath, and restored it to Judah, after that the king slept with his fathers."],
['2Kings','2Ki','14','023','23',"In the fifteenth year of Amaziah the son of Joash king of Judah Jeroboam the son of Joash king of Israel began to reign in Samaria, \emph{and} \emph{reigned} forty and one years."],
['2Kings','2Ki','14','024','24',"And he did \emph{that} \emph{which} \emph{was} evil in the sight of the LORD: he departed not from all the sins of Jeroboam the son of Nebat, who made Israel to sin."],
['2Kings','2Ki','14','025','25',"He restored the coast of Israel from the entering of Hamath unto the sea of the plain, according to the word of the LORD God of Israel, which he spake by the hand of his servant Jonah, the son of Amittai, the prophet, which \emph{was} of Gath-hepher."],
['2Kings','2Ki','14','026','26',"For the LORD saw the affliction of Israel, \emph{that} \emph{it} \emph{was} very bitter: for \emph{there} \emph{was} not any shut up, nor any left, nor any helper for Israel."],
['2Kings','2Ki','14','027','27',"And the LORD said not that he would blot out the name of Israel from under heaven: but he saved them by the hand of Jeroboam the son of Joash."],
['2Kings','2Ki','14','028','28',"Now the rest of the acts of Jeroboam, and all that he did, and his might, how he warred, and how he recovered Damascus, and Hamath, \emph{which} \emph{belonged} to Judah, for Israel, are they not written in the book of the chronicles of the kings of Israel?"],
['2Kings','2Ki','14','029','29',"And Jeroboam slept with his fathers, \emph{even} with the kings of Israel; and Zachariah his son reigned in his stead."]
]

SecondKings15 = [
    ['2Kings','2Ki','15','001','1',"In the twenty and seventh year of Jeroboam king of Israel began Azariah son of Amaziah king of Judah to reign."],
['2Kings','2Ki','15','002','2',"Sixteen years old was he when he began to reign, and he reigned two and fifty years in Jerusalem. And his mother's name \emph{was} Jecholiah of Jerusalem."],
['2Kings','2Ki','15','003','3',"And he did \emph{that} \emph{which} \emph{was} right in the sight of the LORD, according to all that his father Amaziah had done;"],
['2Kings','2Ki','15','004','4',"Save that the high places were not removed: the people sacrificed and burnt incense still on the high places."],
['2Kings','2Ki','15','005','5',"And the LORD smote the king, so that he was a leper unto the day of his death, and dwelt in a several house. And Jotham the king's son \emph{was} over the house, judging the people of the land."],
['2Kings','2Ki','15','006','6',"And the rest of the acts of Azariah, and all that he did, \emph{are} they not written in the book of the chronicles of the kings of Judah?"],
['2Kings','2Ki','15','007','7',"So Azariah slept with his fathers; and they buried him with his fathers in the city of David: and Jotham his son reigned in his stead."],
['2Kings','2Ki','15','008','8',"In the thirty and eighth year of Azariah king of Judah did Zachariah the son of Jeroboam reign over Israel in Samaria six months."],
['2Kings','2Ki','15','009','9',"And he did \emph{that} \emph{which} \emph{was} evil in the sight of the LORD, as his fathers had done: he departed not from the sins of Jeroboam the son of Nebat, who made Israel to sin."],
['2Kings','2Ki','15','010','10',"And Shallum the son of Jabesh conspired against him, and smote him before the people, and slew him, and reigned in his stead."],
['2Kings','2Ki','15','011','11',"And the rest of the acts of Zachariah, behold, they \emph{are} written in the book of the chronicles of the kings of Israel."],
['2Kings','2Ki','15','012','12',"This \emph{was} the word of the LORD which he spake unto Jehu, saying, Thy sons shall sit on the throne of Israel unto the fourth \emph{generation}. And so it came to pass."],
['2Kings','2Ki','15','013','13',"Shallum the son of Jabesh began to reign in the nine and thirtieth year of Uzziah king of Judah; and he reigned a full month in Samaria."],
['2Kings','2Ki','15','014','14',"For Menahem the son of Gadi went up from Tirzah, and came to Samaria, and smote Shallum the son of Jabesh in Samaria, and slew him, and reigned in his stead."],
['2Kings','2Ki','15','015','15',"And the rest of the acts of Shallum, and his conspiracy which he made, behold, they \emph{are} written in the book of the chronicles of the kings of Israel."],
['2Kings','2Ki','15','016','16',"Then Menahem smote Tiphsah, and all that \emph{were} therein, and the coasts thereof from Tirzah: because they opened not \emph{to} \emph{him}, therefore he smote \emph{it;} \emph{and} all the women therein that were with child he ripped up."],
['2Kings','2Ki','15','017','17',"In the nine and thirtieth year of Azariah king of Judah began Menahem the son of Gadi to reign over Israel, \emph{and} \emph{reigned} ten years in Samaria."],
['2Kings','2Ki','15','018','18',"And he did \emph{that} \emph{which} \emph{was} evil in the sight of the LORD: he departed not all his days from the sins of Jeroboam the son of Nebat, who made Israel to sin."],
['2Kings','2Ki','15','019','19',"\emph{And} Pul the king of Assyria came against the land: and Menahem gave Pul a thousand talents of silver, that his hand might be with him to confirm the kingdom in his hand."],
['2Kings','2Ki','15','020','20',"And Menahem exacted the money of Israel, \emph{even} of all the mighty men of wealth, of each man fifty shekels of silver, to give to the king of Assyria. So the king of Assyria turned back, and stayed not there in the land."],
['2Kings','2Ki','15','021','21',"And the rest of the acts of Menahem, and all that he did, \emph{are} they not written in the book of the chronicles of the kings of Israel?"],
['2Kings','2Ki','15','022','22',"And Menahem slept with his fathers; and Pekahiah his son reigned in his stead."],
['2Kings','2Ki','15','023','23',"In the fiftieth year of Azariah king of Judah Pekahiah the son of Menahem began to reign over Israel in Samaria, \emph{and} \emph{reigned} two years."],
['2Kings','2Ki','15','024','24',"And he did \emph{that} \emph{which} \emph{was} evil in the sight of the LORD: he departed not from the sins of Jeroboam the son of Nebat, who made Israel to sin."],
['2Kings','2Ki','15','025','25',"But Pekah the son of Remaliah, a captain of his, conspired against him, and smote him in Samaria, in the palace of the king's house, with Argob and Arieh, and with him fifty men of the Gileadites: and he killed him, and reigned in his room."],
['2Kings','2Ki','15','026','26',"And the rest of the acts of Pekahiah, and all that he did, behold, they \emph{are} written in the book of the chronicles of the kings of Israel."],
['2Kings','2Ki','15','027','27',"In the two and fiftieth year of Azariah king of Judah Pekah the son of Remaliah began to reign over Israel in Samaria, \emph{and} \emph{reigned} twenty years."],
['2Kings','2Ki','15','028','28',"And he did \emph{that} \emph{which} \emph{was} evil in the sight of the LORD: he departed not from the sins of Jeroboam the son of Nebat, who made Israel to sin."],
['2Kings','2Ki','15','029','29',"In the days of Pekah king of Israel came Tiglath-pileser king of Assyria, and took Ijon, and Abel-beth-maachah, and Janoah, and Kedesh, and Hazor, and Gilead, and Galilee, all the land of Naphtali, and carried them captive to Assyria."],
['2Kings','2Ki','15','030','30',"And Hoshea the son of Elah made a conspiracy against Pekah the son of Remaliah, and smote him, and slew him, and reigned in his stead, in the twentieth year of Jotham the son of Uzziah."],
['2Kings','2Ki','15','031','31',"And the rest of the acts of Pekah, and all that he did, behold, they \emph{are} written in the book of the chronicles of the kings of Israel."],
['2Kings','2Ki','15','032','32',"In the second year of Pekah the son of Remaliah king of Israel began Jotham the son of Uzziah king of Judah to reign."],
['2Kings','2Ki','15','033','33',"Five and twenty years old was he when he began to reign, and he reigned sixteen years in Jerusalem. And his mother's name \emph{was} Jerusha, the daughter of Zadok."],
['2Kings','2Ki','15','034','34',"And he did \emph{that} \emph{which} \emph{was} right in the sight of the LORD: he did according to all that his father Uzziah had done."],
['2Kings','2Ki','15','035','35',"Howbeit the high places were not removed: the people sacrificed and burned incense still in the high places. He built the higher gate of the house of the LORD."],
['2Kings','2Ki','15','036','36',"Now the rest of the acts of Jotham, and all that he did, \emph{are} they not written in the book of the chronicles of the kings of Judah?"],
['2Kings','2Ki','15','037','37',"In those days the LORD began to send against Judah Rezin the king of Syria, and Pekah the son of Remaliah."],
['2Kings','2Ki','15','038','38',"And Jotham slept with his fathers, and was buried with his fathers in the city of David his father: and Ahaz his son reigned in his stead."]
]

SecondKings16 = [
    ['2Kings','2Ki','16','001','1',"In the seventeenth year of Pekah the son of Remaliah Ahaz the son of Jotham king of Judah began to reign."],
['2Kings','2Ki','16','002','2',"Twenty years old \emph{was} Ahaz when he began to reign, and reigned sixteen years in Jerusalem, and did not \emph{that} \emph{which} \emph{was} right in the sight of the LORD his God, like David his father."],
['2Kings','2Ki','16','003','3',"But he walked in the way of the kings of Israel, yea, and made his son to pass through the fire, according to the abominations of the heathen, whom the LORD cast out from before the children of Israel."],
['2Kings','2Ki','16','004','4',"And he sacrificed and burnt incense in the high places, and on the hills, and under every green tree."],
['2Kings','2Ki','16','005','5',"Then Rezin king of Syria and Pekah son of Remaliah king of Israel came up to Jerusalem to war: and they besieged Ahaz, but could not overcome \emph{him}."],
['2Kings','2Ki','16','006','6',"At that time Rezin king of Syria recovered Elath to Syria, and drave the Jews from Elath: and the Syrians came to Elath, and dwelt there unto this day."],
['2Kings','2Ki','16','007','7',"So Ahaz sent messengers to Tiglath-pileser king of Assyria, saying, I \emph{am} thy servant and thy son: come up, and save me out of the hand of the king of Syria, and out of the hand of the king of Israel, which rise up against me."],
['2Kings','2Ki','16','008','8',"And Ahaz took the silver and gold that was found in the house of the LORD, and in the treasures of the king's house, and sent \emph{it} \emph{for} a present to the king of Assyria."],
['2Kings','2Ki','16','009','9',"And the king of Assyria hearkened unto him: for the king of Assyria went up against Damascus, and took it, and carried \emph{the} \emph{people} \emph{of} it captive to Kir, and slew Rezin."],
['2Kings','2Ki','16','010','10',"And king Ahaz went to Damascus to meet Tiglath-pileser king of Assyria, and saw an altar that \emph{was} at Damascus: and king Ahaz sent to Urijah the priest the fashion of the altar, and the pattern of it, according to all the workmanship thereof."],
['2Kings','2Ki','16','011','11',"And Urijah the priest built an altar according to all that king Ahaz had sent from Damascus: so Urijah the priest made \emph{it} against king Ahaz came from Damascus."],
['2Kings','2Ki','16','012','12',"And when the king was come from Damascus, the king saw the altar: and the king approached to the altar, and offered thereon."],
['2Kings','2Ki','16','013','13',"And he burnt his burnt offering and his meat offering, and poured his drink offering, and sprinkled the blood of his peace offerings, upon the altar."],
['2Kings','2Ki','16','014','14',"And he brought also the brasen altar, which \emph{was} before the LORD, from the forefront of the house, from between the altar and the house of the LORD, and put it on the north side of the altar."],
['2Kings','2Ki','16','015','15',"And king Ahaz commanded Urijah the priest, saying, Upon the great altar burn the morning burnt offering, and the evening meat offering, and the king's burnt sacrifice, and his meat offering, with the burnt offering of all the people of the land, and their meat offering, and their drink offerings; and sprinkle upon it all the blood of the burnt offering, and all the blood of the sacrifice: and the brasen altar shall be for me to enquire \emph{by}."],
['2Kings','2Ki','16','016','16',"Thus did Urijah the priest, according to all that king Ahaz commanded."],
['2Kings','2Ki','16','017','17',"And king Ahaz cut off the borders of the bases, and removed the laver from off them; and took down the sea from off the brasen oxen that \emph{were} under it, and put it upon a pavement of stones."],
['2Kings','2Ki','16','018','18',"And the covert for the sabbath that they had built in the house, and the king's entry without, turned he from the house of the LORD for the king of Assyria."],
['2Kings','2Ki','16','019','19',"Now the rest of the acts of Ahaz which he did, \emph{are} they not written in the book of the chronicles of the kings of Judah?"],
['2Kings','2Ki','16','020','20',"And Ahaz slept with his fathers, and was buried with his fathers in the city of David: and Hezekiah his son reigned in his stead."]    
]

SecondKings17 = [
    ['2Kings','2Ki','17','001','1',"In the twelfth year of Ahaz king of Judah began Hoshea the son of Elah to reign in Samaria over Israel nine years."],
['2Kings','2Ki','17','002','2',"And he did \emph{that} \emph{which} \emph{was} evil in the sight of the LORD, but not as the kings of Israel that were before him."],
['2Kings','2Ki','17','003','3',"Against him came up Shalmaneser king of Assyria; and Hoshea became his servant, and gave him presents."],
['2Kings','2Ki','17','004','4',"And the king of Assyria found conspiracy in Hoshea: for he had sent messengers to So king of Egypt, and brought no present to the king of Assyria, as \emph{he} \emph{had} \emph{done} year by year: therefore the king of Assyria shut him up, and bound him in prison."],
['2Kings','2Ki','17','005','5',"Then the king of Assyria came up throughout all the land, and went up to Samaria, and besieged it three years."],
['2Kings','2Ki','17','006','6',"In the ninth year of Hoshea the king of Assyria took Samaria, and carried Israel away into Assyria, and placed them in Halah and in Habor \emph{by} the river of Gozan, and in the cities of the Medes."],
['2Kings','2Ki','17','007','7',"For \emph{so} it was, that the children of Israel had sinned against the LORD their God, which had brought them up out of the land of Egypt, from under the hand of Pharaoh king of Egypt, and had feared other gods,"],
['2Kings','2Ki','17','008','8',"And walked in the statutes of the heathen, whom the LORD cast out from before the children of Israel, and of the kings of Israel, which they had made."],
['2Kings','2Ki','17','009','9',"And the children of Israel did secretly \emph{those} things that \emph{were} not right against the LORD their God, and they built them high places in all their cities, from the tower of the watchmen to the fenced city."],
['2Kings','2Ki','17','010','10',"And they set them up images and groves in every high hill, and under every green tree:"],
['2Kings','2Ki','17','011','11',"And there they burnt incense in all the high places, as \emph{did} the heathen whom the LORD carried away before them; and wrought wicked things to provoke the LORD to anger:"],
['2Kings','2Ki','17','012','12',"For they served idols, whereof the LORD had said unto them, Ye shall not do this thing."],
['2Kings','2Ki','17','013','13',"Yet the LORD testified against Israel, and against Judah, by all the prophets, \emph{and} \emph{by} all the seers, saying, Turn ye from your evil ways, and keep my commandments \emph{and} my statutes, according to all the law which I commanded your fathers, and which I sent to you by my servants the prophets."],
['2Kings','2Ki','17','014','14',"Notwithstanding they would not hear, but hardened their necks, like to the neck of their fathers, that did not believe in the LORD their God."],
['2Kings','2Ki','17','015','15',"And they rejected his statutes, and his covenant that he made with their fathers, and his testimonies which he testified against them; and they followed vanity, and became vain, and went after the heathen that \emph{were} round about them, \emph{concerning} whom the LORD had charged them, that they should not do like them."],
['2Kings','2Ki','17','016','16',"And they left all the commandments of the LORD their God, and made them molten images, \emph{even} two calves, and made a grove, and worshipped all the host of heaven, and served Baal."],
['2Kings','2Ki','17','017','17',"And they caused their sons and their daughters to pass through the fire, and used divination and enchantments, and sold themselves to do evil in the sight of the LORD, to provoke him to anger."],
['2Kings','2Ki','17','018','18',"Therefore the LORD was very angry with Israel, and removed them out of his sight: there was none left but the tribe of Judah only."],
['2Kings','2Ki','17','019','19',"Also Judah kept not the commandments of the LORD their God, but walked in the statutes of Israel which they made."],
['2Kings','2Ki','17','020','20',"And the LORD rejected all the seed of Israel, and afflicted them, and delivered them into the hand of spoilers, until he had cast them out of his sight."],
['2Kings','2Ki','17','021','21',"For he rent Israel from the house of David; and they made Jeroboam the son of Nebat king: and Jeroboam drave Israel from following the LORD, and made them sin a great sin."],
['2Kings','2Ki','17','022','22',"For the children of Israel walked in all the sins of Jeroboam which he did; they departed not from them;"],
['2Kings','2Ki','17','023','23',"Until the LORD removed Israel out of his sight, as he had said by all his servants the prophets. So was Israel carried away out of their own land to Assyria unto this day."],
['2Kings','2Ki','17','024','24',"And the king of Assyria brought \emph{men} from Babylon, and from Cuthah, and from Ava, and from Hamath, and from Sepharvaim, and placed \emph{them} in the cities of Samaria instead of the children of Israel: and they possessed Samaria, and dwelt in the cities thereof."],
['2Kings','2Ki','17','025','25',"And \emph{so} it was at the beginning of their dwelling there, \emph{that} they feared not the LORD: therefore the LORD sent lions among them, which slew \emph{some} of them."],
['2Kings','2Ki','17','026','26',"Wherefore they spake to the king of Assyria, saying, The nations which thou hast removed, and placed in the cities of Samaria, know not the manner of the God of the land: therefore he hath sent lions among them, and, behold, they slay them, because they know not the manner of the God of the land."],
['2Kings','2Ki','17','027','27',"Then the king of Assyria commanded, saying, Carry thither one of the priests whom ye brought from thence; and let them go and dwell there, and let him teach them the manner of the God of the land."],
['2Kings','2Ki','17','028','28',"Then one of the priests whom they had carried away from Samaria came and dwelt in Beth-el, and taught them how they should fear the LORD."],
['2Kings','2Ki','17','029','29',"Howbeit every nation made gods of their own, and put \emph{them} in the houses of the high places which the Samaritans had made, every nation in their cities wherein they dwelt."],
['2Kings','2Ki','17','030','30',"And the men of Babylon made Succoth-benoth, and the men of Cuth made Nergal, and the men of Hamath made Ashima,"],
['2Kings','2Ki','17','031','31',"And the Avites made Nibhaz and Tartak, and the Sepharvites burnt their children in fire to Adrammelech and Anammelech, the gods of Sepharvaim."],
['2Kings','2Ki','17','032','32',"So they feared the LORD, and made unto themselves of the lowest of them priests of the high places, which sacrificed for them in the houses of the high places."],
['2Kings','2Ki','17','033','33',"They feared the LORD, and served their own gods, after the manner of the nations whom they carried away from thence."],
['2Kings','2Ki','17','034','34',"Unto this day they do after the former manners: they fear not the LORD, neither do they after their statutes, or after their ordinances, or after the law and commandment which the LORD commanded the children of Jacob, whom he named Israel;"],
['2Kings','2Ki','17','035','35',"With whom the LORD had made a covenant, and charged them, saying, Ye shall not fear other gods, nor bow yourselves to them, nor serve them, nor sacrifice to them:"],
['2Kings','2Ki','17','036','36',"But the LORD, who brought you up out of the land of Egypt with great power and a stretched out arm, him shall ye fear, and him shall ye worship, and to him shall ye do sacrifice."],
['2Kings','2Ki','17','037','37',"And the statutes, and the ordinances, and the law, and the commandment, which he wrote for you, ye shall observe to do for evermore; and ye shall not fear other gods."],
['2Kings','2Ki','17','038','38',"And the covenant that I have made with you ye shall not forget; neither shall ye fear other gods."],
['2Kings','2Ki','17','039','39',"But the LORD your God ye shall fear; and he shall deliver you out of the hand of all your enemies."],
['2Kings','2Ki','17','040','40',"Howbeit they did not hearken, but they did after their former manner."],
['2Kings','2Ki','17','041','41',"So these nations feared the LORD, and served their graven images, both their children, and their children's children: as did their fathers, so do they unto this day."]
]

SecondKings18 = [
    ['2Kings','2Ki','18','001','1',"Now it came to pass in the third year of Hoshea son of Elah king of Israel, \emph{that} Hezekiah the son of Ahaz king of Judah began to reign."],
['2Kings','2Ki','18','002','2',"Twenty and five years old was he when he began to reign; and he reigned twenty and nine years in Jerusalem. His mother's name also \emph{was} Abi, the daughter of Zachariah."],
['2Kings','2Ki','18','003','3',"And he did \emph{that} \emph{which} \emph{was} right in the sight of the LORD, according to all that David his father did."],
['2Kings','2Ki','18','004','4',"He removed the high places, and brake the images, and cut down the groves, and brake in pieces the brasen serpent that Moses had made: for unto those days the children of Israel did burn incense to it: and he called it Nehushtan."],
['2Kings','2Ki','18','005','5',"He trusted in the LORD God of Israel; so that after him was none like him among all the kings of Judah, nor \emph{any} that were before him."],
['2Kings','2Ki','18','006','6',"For he clave to the LORD, \emph{and} departed not from following him, but kept his commandments, which the LORD commanded Moses."],
['2Kings','2Ki','18','007','7',"And the LORD was with him; \emph{and} he prospered whithersoever he went forth: and he rebelled against the king of Assyria, and served him not."],
['2Kings','2Ki','18','008','8',"He smote the Philistines, \emph{even} unto Gaza, and the borders thereof, from the tower of the watchmen to the fenced city."],
['2Kings','2Ki','18','009','9',"And it came to pass in the fourth year of king Hezekiah, which \emph{was} the seventh year of Hoshea son of Elah king of Israel, \emph{that} Shalmaneser king of Assyria came up against Samaria, and besieged it."],
['2Kings','2Ki','18','010','10',"And at the end of three years they took it: \emph{even} in the sixth year of Hezekiah, that \emph{is} the ninth year of Hoshea king of Israel, Samaria was taken."],
['2Kings','2Ki','18','011','11',"And the king of Assyria did carry away Israel unto Assyria, and put them in Halah and in Habor \emph{by} the river of Gozan, and in the cities of the Medes:"],
['2Kings','2Ki','18','012','12',"Because they obeyed not the voice of the LORD their God, but transgressed his covenant, \emph{and} all that Moses the servant of the LORD commanded, and would not hear \emph{them}, nor do \emph{them}."],
['2Kings','2Ki','18','013','13',"Now in the fourteenth year of king Hezekiah did Sennacherib king of Assyria come up against all the fenced cities of Judah, and took them."],
['2Kings','2Ki','18','014','14',"And Hezekiah king of Judah sent to the king of Assyria to Lachish, saying, I have offended; return from me: that which thou puttest on me will I bear. And the king of Assyria appointed unto Hezekiah king of Judah three hundred talents of silver and thirty talents of gold."],
['2Kings','2Ki','18','015','15',"And Hezekiah gave \emph{him} all the silver that was found in the house of the LORD, and in the treasures of the king's house."],
['2Kings','2Ki','18','016','16',"At that time did Hezekiah cut off \emph{the} \emph{gold} \emph{from} the doors of the temple of the LORD, and \emph{from} the pillars which Hezekiah king of Judah had overlaid, and gave it to the king of Assyria."],
['2Kings','2Ki','18','017','17',"And the king of Assyria sent Tartan and Rabsaris and Rabshakeh from Lachish to king Hezekiah with a great host against Jerusalem. And they went up and came to Jerusalem. And when they were come up, they came and stood by the conduit of the upper pool, which \emph{is} in the highway of the fuller's field."],
['2Kings','2Ki','18','018','18',"And when they had called to the king, there came out to them Eliakim the son of Hilkiah, which \emph{was} over the household, and Shebna the scribe, and Joah the son of Asaph the recorder."],
['2Kings','2Ki','18','019','19',"And Rabshakeh said unto them, Speak ye now to Hezekiah, Thus saith the great king, the king of Assyria, What confidence \emph{is} this wherein thou trustest?"],
['2Kings','2Ki','18','020','20',"Thou sayest, (but \emph{they} \emph{are} \emph{but} vain words,) \emph{I} \emph{have} counsel and strength for the war. Now on whom dost thou trust, that thou rebellest against me?"],
['2Kings','2Ki','18','021','21',"Now, behold, thou trustest upon the staff of this bruised reed, \emph{even} upon Egypt, on which if a man lean, it will go into his hand, and pierce it: so \emph{is} Pharaoh king of Egypt unto all that trust on him."],
['2Kings','2Ki','18','022','22',"But if ye say unto me, We trust in the LORD our God: \emph{is} not that he, whose high places and whose altars Hezekiah hath taken away, and hath said to Judah and Jerusalem, Ye shall worship before this altar in Jerusalem?"],
['2Kings','2Ki','18','023','23',"Now therefore, I pray thee, give pledges to my lord the king of Assyria, and I will deliver thee two thousand horses, if thou be able on thy part to set riders upon them."],
['2Kings','2Ki','18','024','24',"How then wilt thou turn away the face of one captain of the least of my master's servants, and put thy trust on Egypt for chariots and for horsemen?"],
['2Kings','2Ki','18','025','25',"Am I now come up without the LORD against this place to destroy it? The LORD said to me, Go up against this land, and destroy it."],
['2Kings','2Ki','18','026','26',"Then said Eliakim the son of Hilkiah, and Shebna, and Joah, unto Rabshakeh, Speak, I pray thee, to thy servants in the Syrian language; for we understand \emph{it}: and talk not with us in the Jews' language in the ears of the people that \emph{are} on the wall."],
['2Kings','2Ki','18','027','27',"But Rabshakeh said unto them, Hath my master sent me to thy master, and to thee, to speak these words? \emph{hath} \emph{he} not \emph{sent} \emph{me} to the men which sit on the wall, that they may eat their own dung, and drink their own piss with you?"],
['2Kings','2Ki','18','028','28',"Then Rabshakeh stood and cried with a loud voice in the Jews' language, and spake, saying, Hear the word of the great king, the king of Assyria:"],
['2Kings','2Ki','18','029','29',"Thus saith the king, Let not Hezekiah deceive you: for he shall not be able to deliver you out of his hand:"],
['2Kings','2Ki','18','030','30',"Neither let Hezekiah make you trust in the LORD, saying, The LORD will surely deliver us, and this city shall not be delivered into the hand of the king of Assyria."],
['2Kings','2Ki','18','031','31',"Hearken not to Hezekiah: for thus saith the king of Assyria, Make \emph{an} \emph{agreement} with me by a present, and come out to me, and \emph{then} eat ye every man of his own vine, and every one of his fig tree, and drink ye every one the waters of his cistern:"],
['2Kings','2Ki','18','032','32',"Until I come and take you away to a land like your own land, a land of corn and wine, a land of bread and vineyards, a land of oil olive and of honey, that ye may live, and not die: and hearken not unto Hezekiah, when he persuadeth you, saying, The LORD will deliver us."],
['2Kings','2Ki','18','033','33',"Hath any of the gods of the nations delivered at all his land out of the hand of the king of Assyria?"],
['2Kings','2Ki','18','034','34',"Where \emph{are} the gods of Hamath, and of Arpad? where \emph{are} the gods of Sepharvaim, Hena, and Ivah? have they delivered Samaria out of mine hand?"],
['2Kings','2Ki','18','035','35',"Who \emph{are} they among all the gods of the countries, that have delivered their country out of mine hand, that the LORD should deliver Jerusalem out of mine hand?"],
['2Kings','2Ki','18','036','36',"But the people held their peace, and answered him not a word: for the king's commandment was, saying, Answer him not."],
['2Kings','2Ki','18','037','37',"Then came Eliakim the son of Hilkiah, which \emph{was} over the household, and Shebna the scribe, and Joah the son of Asaph the recorder, to Hezekiah with \emph{their} clothes rent, and told him the words of Rabshakeh."]
]

SecondKings22 =  [
['2Kings','2Ki','22','001','1',"Josiah \emph{was} eight years old when he began to reign, and he reigned thirty and one years in Jerusalem. And his mother's name \emph{was} Jedidah, the daughter of Adaiah of Boscath."],
['2Kings','2Ki','22','002','2',"And he did \emph{that} \emph{which} \emph{was} right in the sight of the LORD, and walked in all the way of David his father, and turned not aside to the right hand or to the left."],
['2Kings','2Ki','22','003','3',"And it came to pass in the eighteenth year of king Josiah, \emph{that} the king sent Shaphan the son of Azaliah, the son of Meshullam, the scribe, to the house of the LORD, saying,"],
['2Kings','2Ki','22','004','4',"Go up to Hilkiah the high priest, that he may sum the silver which is brought into the house of the LORD, which the keepers of the door have gathered of the people:"],
['2Kings','2Ki','22','005','5',"And let them deliver it into the hand of the doers of the work, that have the oversight of the house of the LORD: and let them give it to the doers of the work which \emph{is} in the house of the LORD, to repair the breaches of the house,"],
['2Kings','2Ki','22','006','6',"Unto carpenters, and builders, and masons, and to buy timber and hewn stone to repair the house."],
['2Kings','2Ki','22','007','7',"Howbeit there was no reckoning made with them of the money that was delivered into their hand, because they dealt faithfully."],
['2Kings','2Ki','22','008','8',"And Hilkiah the high priest said unto Shaphan the scribe, I have found the book of the law in the house of the LORD. And Hilkiah gave the book to Shaphan, and he read it."],
['2Kings','2Ki','22','009','9',"And Shaphan the scribe came to the king, and brought the king word again, and said, Thy servants have gathered the money that was found in the house, and have delivered it into the hand of them that do the work, that have the oversight of the house of the LORD."],
['2Kings','2Ki','22','010','10',"And Shaphan the scribe shewed the king, saying, Hilkiah the priest hath delivered me a book. And Shaphan read it before the king."],
['2Kings','2Ki','22','011','11',"And it came to pass, when the king had heard the words of the book of the law, that he rent his clothes."],
['2Kings','2Ki','22','012','12',"And the king commanded Hilkiah the priest, and Ahikam the son of Shaphan, and Achbor the son of Michaiah, and Shaphan the scribe, and Asahiah a servant of the king's, saying,"],
['2Kings','2Ki','22','013','13',"Go ye, enquire of the LORD for me, and for the people, and for all Judah, concerning the words of this book that is found: for great \emph{is} the wrath of the LORD that is kindled against us, because our fathers have not hearkened unto the words of this book, to do according unto all that which is written concerning us."],
['2Kings','2Ki','22','014','14',"So Hilkiah the priest, and Ahikam, and Achbor, and Shaphan, and Asahiah, went unto Huldah the prophetess, the wife of Shallum the son of Tikvah, the son of Harhas, keeper of the wardrobe; (now she dwelt in Jerusalem in the college;) and they communed with her."],
['2Kings','2Ki','22','015','15',"And she said unto them, Thus saith the LORD God of Israel, Tell the man that sent you to me,"],
['2Kings','2Ki','22','016','16',"Thus saith the LORD, Behold, I will bring evil upon this place, and upon the inhabitants thereof, \emph{even} all the words of the book which the king of Judah hath read:"],
['2Kings','2Ki','22','017','17',"Because they have forsaken me, and have burned incense unto other gods, that they might provoke me to anger with all the works of their hands; therefore my wrath shall be kindled against this place, and shall not be quenched."],
['2Kings','2Ki','22','018','18',"But to the king of Judah which sent you to enquire of the LORD, thus shall ye say to him, Thus saith the LORD God of Israel, \emph{As} \emph{touching} the words which thou hast heard;"],
['2Kings','2Ki','22','019','19',"Because thine heart was tender, and thou hast humbled thyself before the LORD, when thou heardest what I spake against this place, and against the inhabitants thereof, that they should become a desolation and a curse, and hast rent thy clothes, and wept before me; I also have heard \emph{thee}, saith the LORD."],
['2Kings','2Ki','22','020','20',"Behold therefore, I will gather thee unto thy fathers, and thou shalt be gathered into thy grave in peace; and thine eyes shall not see all the evil which I will bring upon this place. And they brought the king word again."]
]

SecondKings23 = [
['2Kings','2Ki','23','001','1',"And the king sent, and they gathered unto him all the elders of Judah and of Jerusalem."],
['2Kings','2Ki','23','002','2',"And the king went up into the house of the LORD, and all the men of Judah and all the inhabitants of Jerusalem with him, and the priests, and the prophets, and all the people, both small and great: and he read in their ears all the words of the book of the covenant which was found in the house of the LORD."],
['2Kings','2Ki','23','003','3',"And the king stood by a pillar, and made a covenant before the LORD, to walk after the LORD, and to keep his commandments and his testimonies and his statutes with all \emph{their} heart and all \emph{their} soul, to perform the words of this covenant that were written in this book. And all the people stood to the covenant."],
['2Kings','2Ki','23','004','4',"And the king commanded Hilkiah the high priest, and the priests of the second order, and the keepers of the door, to bring forth out of the temple of the LORD all the vessels that were made for Baal, and for the grove, and for all the host of heaven: and he burned them without Jerusalem in the fields of Kidron, and carried the ashes of them unto Beth-el."],
['2Kings','2Ki','23','005','5',"And he put down the idolatrous priests, whom the kings of Judah had ordained to burn incense in the high places in the cities of Judah, and in the places round about Jerusalem; them also that burned incense unto Baal, to the sun, and to the moon, and to the planets, and to all the host of heaven."],
['2Kings','2Ki','23','006','6',"And he brought out the grove from the house of the LORD, without Jerusalem, unto the brook Kidron, and burned it at the brook Kidron, and stamped \emph{it} small to powder, and cast the powder thereof upon the graves of the children of the people."],
['2Kings','2Ki','23','007','7',"And he brake down the houses of the sodomites, that \emph{were} by the house of the LORD, where the women wove hangings for the grove."],
['2Kings','2Ki','23','008','8',"And he brought all the priests out of the cities of Judah, and defiled the high places where the priests had burned incense, from Geba to Beer-sheba, and brake down the high places of the gates that \emph{were} in the entering in of the gate of Joshua the governor of the city, which \emph{were} on a man's left hand at the gate of the city."],
['2Kings','2Ki','23','009','9',"Nevertheless the priests of the high places came not up to the altar of the LORD in Jerusalem, but they did eat of the unleavened bread among their brethren."],
['2Kings','2Ki','23','010','10',"And he defiled Topheth, which \emph{is} in the valley of the children of Hinnom, that no man might make his son or his daughter to pass through the fire to Molech."],
['2Kings','2Ki','23','011','11',"And he took away the horses that the kings of Judah had given to the sun, at the entering in of the house of the LORD, by the chamber of Nathan-melech the chamberlain, which \emph{was} in the suburbs, and burned the chariots of the sun with fire."],
['2Kings','2Ki','23','012','12',"And the altars that \emph{were} on the top of the upper chamber of Ahaz, which the kings of Judah had made, and the altars which Manasseh had made in the two courts of the house of the LORD, did the king beat down, and brake \emph{them} down from thence, and cast the dust of them into the brook Kidron."],
['2Kings','2Ki','23','013','13',"And the high places that \emph{were} before Jerusalem, which \emph{were} on the right hand of the mount of corruption, which Solomon the king of Israel had builded for Ashtoreth the abomination of the Zidonians, and for Chemosh the abomination of the Moabites, and for Milcom the abomination of the children of Ammon, did the king defile."],
['2Kings','2Ki','23','014','14',"And he brake in pieces the images, and cut down the groves, and filled their places with the bones of men."],
['2Kings','2Ki','23','015','15',"Moreover the altar that \emph{was} at Beth-el, \emph{and} the high place which Jeroboam the son of Nebat, who made Israel to sin, had made, both that altar and the high place he brake down, and burned the high place, \emph{and} stamped \emph{it} small to powder, and burned the grove."],
['2Kings','2Ki','23','016','16',"And as Josiah turned himself, he spied the sepulchres that \emph{were} there in the mount, and sent, and took the bones out of the sepulchres, and burned \emph{them} upon the altar, and polluted it, according to the word of the LORD which the man of God proclaimed, who proclaimed these words."],
['2Kings','2Ki','23','017','17',"Then he said, What title \emph{is} that that I see? And the men of the city told him, \emph{It} \emph{is} the sepulchre of the man of God, which came from Judah, and proclaimed these things that thou hast done against the altar of Beth-el."],
['2Kings','2Ki','23','018','18',"And he said, Let him alone; let no man move his bones. So they let his bones alone, with the bones of the prophet that came out of Samaria."],
['2Kings','2Ki','23','019','19',"And all the houses also of the high places that \emph{were} in the cities of Samaria, which the kings of Israel had made to provoke \emph{the} \emph{LORD} to anger, Josiah took away, and did to them according to all the acts that he had done in Beth-el."],
['2Kings','2Ki','23','020','20',"And he slew all the priests of the high places that \emph{were} there upon the altars, and burned men's bones upon them, and returned to Jerusalem."],
['2Kings','2Ki','23','021','21',"And the king commanded all the people, saying, Keep the passover unto the LORD your God, as \emph{it} \emph{is} written in the book of this covenant."],
['2Kings','2Ki','23','022','22',"Surely there was not holden such a passover from the days of the judges that judged Israel, nor in all the days of the kings of Israel, nor of the kings of Judah;"],
['2Kings','2Ki','23','023','23',"But in the eighteenth year of king Josiah, \emph{wherein} this passover was holden to the LORD in Jerusalem."],
['2Kings','2Ki','23','024','24',"Moreover the \emph{workers} \emph{with} familiar spirits, and the wizards, and the images, and the idols, and all the abominations that were spied in the land of Judah and in Jerusalem, did Josiah put away, that he might perform the words of the law which were written in the book that Hilkiah the priest found in the house of the LORD."],
['2Kings','2Ki','23','025','25',"And like unto him was there no king before him, that turned to the LORD with all his heart, and with all his soul, and with all his might, according to all the law of Moses; neither after him arose there \emph{any} like him."],
['2Kings','2Ki','23','026','26',"Notwithstanding the LORD turned not from the fierceness of his great wrath, wherewith his anger was kindled against Judah, because of all the provocations that Manasseh had provoked him withal."],
['2Kings','2Ki','23','027','27',"And the LORD said, I will remove Judah also out of my sight, as I have removed Israel, and will cast off this city Jerusalem which I have chosen, and the house of which I said, My name shall be there."],
['2Kings','2Ki','23','028','28',"Now the rest of the acts of Josiah, and all that he did, \emph{are} they not written in the book of the chronicles of the kings of Judah?"],
['2Kings','2Ki','23','029','29',"In his days Pharaoh-nechoh king of Egypt went up against the king of Assyria to the river Euphrates: and king Josiah went against him; and he slew him at Megiddo, when he had seen him."],
['2Kings','2Ki','23','030','30',"And his servants carried him in a chariot dead from Megiddo, and brought him to Jerusalem, and buried him in his own sepulchre. And the people of the land took Jehoahaz the son of Josiah, and anointed him, and made him king in his father's stead."],
['2Kings','2Ki','23','031','31',"Jehoahaz \emph{was} twenty and three years old when he began to reign; and he reigned three months in Jerusalem. And his mother's name \emph{was} Hamutal, the daughter of Jeremiah of Libnah."],
['2Kings','2Ki','23','032','32',"And he did \emph{that} \emph{which} \emph{was} evil in the sight of the LORD, according to all that his fathers had done."],
['2Kings','2Ki','23','033','33',"And Pharaoh-nechoh put him in bands at Riblah in the land of Hamath, that he might not reign in Jerusalem; and put the land to a tribute of an hundred talents of silver, and a talent of gold."],
['2Kings','2Ki','23','034','34',"And Pharaoh-nechoh made Eliakim the son of Josiah king in the room of Josiah his father, and turned his name to Jehoiakim, and took Jehoahaz away: and he came to Egypt, and died there."],
['2Kings','2Ki','23','035','35',"And Jehoiakim gave the silver and the gold to Pharaoh; but he taxed the land to give the money according to the commandment of Pharaoh: he exacted the silver and the gold of the people of the land, of every one according to his taxation, to give \emph{it} unto Pharaoh-nechoh."],
['2Kings','2Ki','23','036','36',"Jehoiakim \emph{was} twenty and five years old when he began to reign; and he reigned eleven years in Jerusalem. And his mother's name \emph{was} Zebudah, the daughter of Pedaiah of Rumah."],
['2Kings','2Ki','23','037','37',"And he did \emph{that} \emph{which} \emph{was} evil in the sight of the LORD, according to all that his fathers had done."]
]

SecondKings24 = [
['2Kings','2Ki','24','001','1',"In his days Nebuchadnezzar king of Babylon came up, and Jehoiakim became his servant three years: then he turned and rebelled against him."],
['2Kings','2Ki','24','002','2',"And the LORD sent against him bands of the Chaldees, and bands of the Syrians, and bands of the Moabites, and bands of the children of Ammon, and sent them against Judah to destroy it, according to the word of the LORD, which he spake by his servants the prophets."],
['2Kings','2Ki','24','003','3',"Surely at the commandment of the LORD came \emph{this} upon Judah, to remove \emph{them} out of his sight, for the sins of Manasseh, according to all that he did;"],
['2Kings','2Ki','24','004','4',"And also for the innocent blood that he shed: for he filled Jerusalem with innocent blood; which the LORD would not pardon."],
['2Kings','2Ki','24','005','5',"Now the rest of the acts of Jehoiakim, and all that he did, \emph{are} they not written in the book of the chronicles of the kings of Judah?"],
['2Kings','2Ki','24','006','6',"So Jehoiakim slept with his fathers: and Jehoiachin his son reigned in his stead."],
['2Kings','2Ki','24','007','7',"And the king of Egypt came not again any more out of his land: for the king of Babylon had taken from the river of Egypt unto the river Euphrates all that pertained to the king of Egypt."],
['2Kings','2Ki','24','008','8',"Jehoiachin \emph{was} eighteen years old when he began to reign, and he reigned in Jerusalem three months. And his mother's name \emph{was} Nehushta, the daughter of Elnathan of Jerusalem."],
['2Kings','2Ki','24','009','9',"And he did \emph{that} \emph{which} \emph{was} evil in the sight of the LORD, according to all that his father had done."],
['2Kings','2Ki','24','010','10',"At that time the servants of Nebuchadnezzar king of Babylon came up against Jerusalem, and the city was besieged."],
['2Kings','2Ki','24','011','11',"And Nebuchadnezzar king of Babylon came against the city, and his servants did besiege it."],
['2Kings','2Ki','24','012','12',"And Jehoiachin the king of Judah went out to the king of Babylon, he, and his mother, and his servants, and his princes, and his officers: and the king of Babylon took him in the eighth year of his reign."],
['2Kings','2Ki','24','013','13',"And he carried out thence all the treasures of the house of the LORD, and the treasures of the king's house, and cut in pieces all the vessels of gold which Solomon king of Israel had made in the temple of the LORD, as the LORD had said."],
['2Kings','2Ki','24','014','14',"And he carried away all Jerusalem, and all the princes, and all the mighty men of valour, \emph{even} ten thousand captives, and all the craftsmen and smiths: none remained, save the poorest sort of the people of the land."],
['2Kings','2Ki','24','015','15',"And he carried away Jehoiachin to Babylon, and the king's mother, and the king's wives, and his officers, and the mighty of the land, \emph{those} carried he into captivity from Jerusalem to Babylon."],
['2Kings','2Ki','24','016','16',"And all the men of might, \emph{even} seven thousand, and craftsmen and smiths a thousand, all \emph{that} \emph{were} strong \emph{and} apt for war, even them the king of Babylon brought captive to Babylon."],
['2Kings','2Ki','24','017','17',"And the king of Babylon made Mattaniah his father's brother king in his stead, and changed his name to Zedekiah."],
['2Kings','2Ki','24','018','18',"Zedekiah \emph{was} twenty and one years old when he began to reign, and he reigned eleven years in Jerusalem. And his mother's name \emph{was} Hamutal, the daughter of Jeremiah of Libnah."],
['2Kings','2Ki','24','019','19',"And he did \emph{that} \emph{which} \emph{was} evil in the sight of the LORD, according to all that Jehoiakim had done."],
['2Kings','2Ki','24','020','20',"For through the anger of the LORD it came to pass in Jerusalem and Judah, until he had cast them out from his presence, that Zedekiah rebelled against the king of Babylon."]    
]

SecondKings25 = [
['2Kings','2Ki','25','001','1',"And it came to pass in the ninth year of his reign, in the tenth month, in the tenth \emph{day} of the month, \emph{that} Nebuchadnezzar king of Babylon came, he, and all his host, against Jerusalem, and pitched against it; and they built forts against it round about."],
['2Kings','2Ki','25','002','2',"And the city was besieged unto the eleventh year of king Zedekiah."],
['2Kings','2Ki','25','003','3',"And on the ninth \emph{day} of the \emph{fourth} month the famine prevailed in the city, and there was no bread for the people of the land."],
['2Kings','2Ki','25','004','4',"And the city was broken up, and all the men of war \emph{fled} by night by the way of the gate between two walls, which \emph{is} by the king's garden: (now the Chaldees \emph{were} against the city round about:) and \emph{the} \emph{king} went the way toward the plain."],
['2Kings','2Ki','25','005','5',"And the army of the Chaldees pursued after the king, and overtook him in the plains of Jericho: and all his army were scattered from him."],
['2Kings','2Ki','25','006','6',"So they took the king, and brought him up to the king of Babylon to Riblah; and they gave judgment upon him."],
['2Kings','2Ki','25','007','7',"And they slew the sons of Zedekiah before his eyes, and put out the eyes of Zedekiah, and bound him with fetters of brass, and carried him to Babylon."],
['2Kings','2Ki','25','008','8',"And in the fifth month, on the seventh \emph{day} of the month, which \emph{is} the nineteenth year of king Nebuchadnezzar king of Babylon, came Nebuzar-adan, captain of the guard, a servant of the king of Babylon, unto Jerusalem:"],
['2Kings','2Ki','25','009','9',"And he burnt the house of the LORD, and the king's house, and all the houses of Jerusalem, and every great \emph{man's} house burnt he with fire."],
['2Kings','2Ki','25','010','10',"And all the army of the Chaldees, that \emph{were} \emph{with} the captain of the guard, brake down the walls of Jerusalem round about."],
['2Kings','2Ki','25','011','11',"Now the rest of the people \emph{that} \emph{were} left in the city, and the fugitives that fell away to the king of Babylon, with the remnant of the multitude, did Nebuzar-adan the captain of the guard carry away."],
['2Kings','2Ki','25','012','12',"But the captain of the guard left of the poor of the land \emph{to} \emph{be} vinedressers and husbandmen."],
['2Kings','2Ki','25','013','13',"And the pillars of brass that \emph{were} in the house of the LORD, and the bases, and the brasen sea that \emph{was} in the house of the LORD, did the Chaldees break in pieces, and carried the brass of them to Babylon."],
['2Kings','2Ki','25','014','14',"And the pots, and the shovels, and the snuffers, and the spoons, and all the vessels of brass wherewith they ministered, took they away."],
['2Kings','2Ki','25','015','15',"And the firepans, and the bowls, \emph{and} such things as \emph{were} of gold, \emph{in} gold, and of silver, \emph{in} silver, the captain of the guard took away."],
['2Kings','2Ki','25','016','16',"The two pillars, one sea, and the bases which Solomon had made for the house of the LORD; the brass of all these vessels was without weight."],
['2Kings','2Ki','25','017','17',"The height of the one pillar \emph{was} eighteen cubits, and the chapiter upon it \emph{was} brass: and the height of the chapiter three cubits; and the wreathen work, and pomegranates upon the chapiter round about, all of brass: and like unto these had the second pillar with wreathen work."],
['2Kings','2Ki','25','018','18',"And the captain of the guard took Seraiah the chief priest, and Zephaniah the second priest, and the three keepers of the door:"],
['2Kings','2Ki','25','019','19',"And out of the city he took an officer that was set over the men of war, and five men of them that were in the king's presence, which were found in the city, and the principal scribe of the host, which mustered the people of the land, and threescore men of the people of the land \emph{that} \emph{were} found in the city:"],
['2Kings','2Ki','25','020','20',"And Nebuzar-adan captain of the guard took these, and brought them to the king of Babylon to Riblah:"],
['2Kings','2Ki','25','021','21',"And the king of Babylon smote them, and slew them at Riblah in the land of Hamath. So Judah was carried away out of their land."],
['2Kings','2Ki','25','022','22',"And \emph{as} \emph{for} the people that remained in the land of Judah, whom Nebuchadnezzar king of Babylon had left, even over them he made Gedaliah the son of Ahikam, the son of Shaphan, ruler."],
['2Kings','2Ki','25','023','23',"And when all the captains of the armies, they and their men, heard that the king of Babylon had made Gedaliah governor, there came to Gedaliah to Mizpah, even Ishmael the son of Nethaniah, and Johanan the son of Careah, and Seraiah the son of Tanhumeth the Netophathite, and Jaazaniah the son of a Maachathite, they and their men."],
['2Kings','2Ki','25','024','24',"And Gedaliah sware to them, and to their men, and said unto them, Fear not to be the servants of the Chaldees: dwell in the land, and serve the king of Babylon; and it shall be well with you."],
['2Kings','2Ki','25','025','25',"But it came to pass in the seventh month, that Ishmael the son of Nethaniah, the son of Elishama, of the seed royal, came, and ten men with him, and smote Gedaliah, that he died, and the Jews and the Chaldees that were with him at Mizpah."],
['2Kings','2Ki','25','026','26',"And all the people, both small and great, and the captains of the armies, arose, and came to Egypt: for they were afraid of the Chaldees."],
['2Kings','2Ki','25','027','27',"And it came to pass in the seven and thirtieth year of the captivity of Jehoiachin king of Judah, in the twelfth month, on the seven and twentieth \emph{day} of the month, \emph{that} Evil-merodach king of Babylon in the year that he began to reign did lift up the head of Jehoiachin king of Judah out of prison;"],
['2Kings','2Ki','25','028','28',"And he spake kindly to him, and set his throne above the throne of the kings that \emph{were} with him in Babylon;"],
['2Kings','2Ki','25','029','29',"And changed his prison garments: and he did eat bread continually before him all the days of his life."],
['2Kings','2Ki','25','030','30',"And his allowance \emph{was} a continual allowance given him of the king, a daily rate for every day, all the days of his life."]
]

def main():
    ProcessVerses('SecondKings25',SecondKings25)

if __name__ == "__main__":
    main()



%%%%%%%%%%%%%%%%%%%%%%%%%%%
Chapter with Latex format
%%%%%%%%%%%%%%%%%%%%%%%%%%%


\footnote{\textcolor[cmyk]{0.99998,1,0,0}{\hyperlink{TOC}{Return to end of Table of Contents.}}}\textcolor[cmyk]{0.99998,1,0,0}{And it came to pass in the ninth year of his reign, in the tenth month, in the tenth \emph{day} of the month, \emph{that} Nebuchadnezzar king of Babylon came, he, and all his host, against Jerusalem, and pitched against it; and they built forts against it round about.}
[2] \textcolor[cmyk]{0.99998,1,0,0}{And the city was besieged unto the eleventh year of king Zedekiah.}
[3] \textcolor[cmyk]{0.99998,1,0,0}{And on the ninth \emph{day} of the \emph{fourth} month the famine prevailed in the city, and there was no bread for the people of the land.}
[4] \textcolor[cmyk]{0.99998,1,0,0}{And the city was broken up, and all the men of war \emph{fled} by night by the way of the gate between two walls, which \emph{is} by the king's garden: (now the Chaldees \emph{were} against the city 

\emph{was} & 4 \\ \hline
\emph{were} & 7 \\ \hline
\emph{with} & 1 \\ \hline
a & 4 \\ \hline
about & 4 \\ \hline
above & 1 \\ \hline
afraid & 1 \\ \hline
after & 1 \\ \hline
against & 4 \\ \hline
all & 12 \\ \hline
allowance & 2 \\ \hline
an & 1 \\ \hline
and & 60 \\ \hline
armies & 2 \\ \hline
army & 3 \\ \hline
arose & 1 \\ \hline
as & 1 \\ \hline
at & 2 \\ \hline
away & 5 \\ \hline
bases & 2 \\ \hline
be & 2 \\ \hline
before & 2 \\ \hline
began & 1 \\ \hline
besieged & 1 \\ \hline
between & 1 \\ \hline
both & 1 \\ \hline
bound & 1 \\ \hline
bowls & 1 \\ \hline
brake & 1 \\ \hline
brasen & 1 \\ \hline
brass & 7 \\ \hline
bread & 2 \\ \hline
break & 1 \\ \hline
broken & 1 \\ \hline
brought & 2 \\ \hline
built & 1 \\ \hline
burnt & 2 \\ \hline
by & 3 \\ \hline
came & 8 \\ \hline
captain & 7 \\ \hline
captains & 2 \\ \hline
captivity & 1 \\ \hline
carried & 3 \\ \hline
carry & 1 \\ \hline
changed & 1 \\ \hline
chapiter & 3 \\ \hline
chief & 1 \\ \hline
city & 8 \\ \hline
continual & 1 \

2Kings 25:20 & And Nebuzar-adan captain of the guard took these, and brought them to the king of Babylon to Riblah: \\ \hline
\end{longtable}






%%%%%%%%%%
%%%%%%%%%%
 
